In [2]:
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
import time
import requests
import math
import logging
import datetime
import os

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
#     outdata = split_table_name(outdata)
    return outdata

## 计算时间差
def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        pass

    # 计算车辆当前保值率
def computer_with_license_month(tar):
    try:
        license_month = tar['license_month']
        if(license_month<=12):
            #tar['keep_value'] = tar['year_1']
            return tar['year_1']
        else:
            year = license_month//12
            #当前年保值率
            keep_max = tar["year_"+str(int(year))]
            #下一年的保值率
            keep_min = tar["year_"+str(int(year+1))]

            #相比于上一年，已经过了几个月
            mon = license_month-12*year
            tem = (keep_max-keep_min)/12

            #tar['keep_value'] = round(keep_max - tem*mon, 4)
            return round(keep_max - tem*mon, 4)
    except:
        return tar["year_16"]

class Logger:       
    def __init__(self, logName, logFile):
        self._logger = logging.getLogger(logName)
        handler = logging.FileHandler(logFile)
        formatter = logging.Formatter('%(asctime)s ********* %(message)s')
        handler.setFormatter(formatter)
        self._logger.addHandler(handler)
        self._logger.setLevel(logging.INFO)

    def log(self, msg):
        if self._logger is not None:
            self._logger.info(msg)


curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/valuation/'
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

logger = Logger('model_service','./log/accurate_valuation_cyp_run_log.log')

logger.log("程序启动.............")


[2020-12-01 14:46:09,886] {<ipython-input-2-18214e15c7d3>:89} INFO - 当前日期: 2020-12-01
[2020-12-01 14:46:09,897] {<ipython-input-2-18214e15c7d3>:85} INFO - 程序启动.............


In [4]:
# curr_date = '2020-10-25'
sql='''
SELECT
    t1.order_id,
    t1.c_goods_id,
    t1.order_price,
    t1.seller_price,
    t1.order_status,
    t1.auction_mode,
    t1.goods_source,
    t1.pay_time,
    t1.source_type,
    t2.c_register_date first_license_date,
    t2.c_rank_desc rank_desc,
    t2.c_rank_level rank_level,
    t2.c_color color_name,
    t2.c_transfer_count tradetimes,
    t2.c_mileage mileage,
    t2.c_car_model_id model_code,
    t2.c_car_model_name model_name,
    t2.c_series_name series_name,
    t2.c_series_id series_code,
    t2.c_brand brand_name,
    t2.c_brand_id brand_code,
    t2.license_simplify,
    t2.c_register_code city_code,
    t2.c_register_name city_name,
    t2.c_register_province_code province_code,
    t2.c_register_province_name province_name,
    t2.c_min_offering_price,
    t2.c_operation_tag purpose,
    t2.c_trade_code trade_code,
    t2.c_base_price,
    t2.c_buy_price,
    t2.c_auction_price,
    t2.c_goods_source,
    t2.c_has_auction,
    t2.c_auction_num,
    t2.c_car_type,
    t2.c_offsite_car,
    t2.c_ca_status,
    t3.guide_price,
    t4.brand_name_,
    t4.series_name_,
    t4.model_name_,
    t2.use_type,
    t1.tenant_name,
    t1.seller_name,
    t5.is_version,
    t6.c_product_name,
    t6.c_product_display_name,
    t6.c_product_type_name,
    t6.c_remark,
    t7.environmental_standards 
FROM
(
    SELECT
        order_id,
        c_goods_id,
        total_price order_price,
        seller_price,
        order_status,
        auction_mode,
        goods_source,
        create_time pay_time,
        tenant_name,
        seller_name,
        source_type 
    FROM
        edw_cyp.dwb_evt_tra_cypf_order_dd
    WHERE
        ds = date_sub("'''+curr_date+'''",1) AND
        c_goods_id IS NOT NULL 
        and delete_tag = 0 
) t1
JOIN
(
    SELECT
        c_id,
        substr(CAST (c_register_date AS STRING),1,10)  c_register_date,
        c_rank_desc,
        c_rank_level,
        c_color,
        c_transfer_count,
        c_mileage,
        c_car_model_id,
        c_car_model_name,
        c_license,
        c_register_code,
        c_register_name,
        c_register_province_code,
        c_register_province_name,
        c_min_offering_price,
        c_operation_tag,
        c_trade_code,
        c_base_price,
        c_buy_price,
        c_auction_price,
        c_series_name,
        c_series_id ,
        c_brand,
        c_brand_id,
        license_simplify,
        use_type,
        c_goods_source,
        c_has_auction,
        c_auction_num,
        c_car_type,
        c_offsite_car,
        c_ca_status
    FROM 
        edw_cyp.dwb_gds_cypd_goods_dd
    WHERE
        ds =  date_sub("'''+curr_date+'''",1)
) t2 
ON t1.c_goods_id = t2.c_id 
left join (select model_code,
                  guide_price 
           from db_data.ods_car_model_model_price 
           where ds =  date_sub("'''+curr_date+'''",1)) t3 
on t2.c_car_model_id = t3.model_code 
left join (select brand_name brand_name_,
                  series_name series_name_,
                  model_code,
                  model_name model_name_ 
           from db_data.ods_car_model_model  
           where ds = date_sub("'''+curr_date+'''",1)) t4  
on t2.c_car_model_id = t4.model_code 
left join(select report_code,
                 product_id,
                 is_version,
                 product_type,
                 product_type_name
          from edw_cyp.dwb_evt_act_cypf_detect_order_dd
          where ds = date_sub("'''+curr_date+'''",1)) t5 
on t2.c_trade_code = t5.report_code 
left join(select c_id,
                 c_product_name,
                 c_product_display_name,
                 c_product_type_name,
                 c_remark
          from dl_cyp.dl_dp_core_iplm_t_product_dd 
          where ds =date_sub("'''+curr_date+'''",1)) t6 
on t6.c_id = t5.product_id  
left join (select model_code,
                  environmental_standards 
           from db_data.ods_car_model_model_parameter 
           where ds = date_sub("'''+curr_date+'''",1) ) t7 
on t2.c_car_model_id = t7.model_code 
WHERE
    t2.c_register_date IS NOT NULL AND
    t2.c_mileage IS NOT NULL 
    
'''
dtype={'city_code':str}
mid_car_cyp_paimai_order = read_from_hive2('mid_car_cyp_paimai_order_dd',sql,dtype)

cyp_paimai_order_columns = []
for col in mid_car_cyp_paimai_order.columns:
    if len(col.split('.')) > 1:
        cyp_paimai_order_columns.append(col.split('.')[1])

    else:
        cyp_paimai_order_columns.append(col)
mid_car_cyp_paimai_order.columns = cyp_paimai_order_columns 
logger.log("读取数据完成.............")


[2020-12-01 14:46:13,836] {hiveserver2:138} INFO - Using database default as default
[2020-12-01 14:46:13,972] {hive_server_hook:112} INFO - Running query: 
SELECT
    t1.order_id,
    t1.c_goods_id,
    t1.order_price,
    t1.seller_price,
    t1.order_status,
    t1.auction_mode,
    t1.goods_source,
    t1.pay_time,
    t1.source_type,
    t2.c_register_date first_license_date,
    t2.c_rank_desc rank_desc,
    t2.c_rank_level rank_level,
    t2.c_color color_name,
    t2.c_transfer_count tradetimes,
    t2.c_mileage mileage,
    t2.c_car_model_id model_code,
    t2.c_car_model_name model_name,
    t2.c_series_name series_name,
    t2.c_series_id series_code,
    t2.c_brand brand_name,
    t2.c_brand_id brand_code,
    t2.license_simplify,
    t2.c_register_code city_code,
    t2.c_register_name city_name,
    t2.c_register_province_code province_code,
    t2.c_register_province_name province_name,
    t2.c_min_offering_price,
    t2.c_operation_tag purpose,
    t2.c_trade_code trad

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0,25,43,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


[2020-12-01 14:49:05,982] {<ipython-input-2-18214e15c7d3>:85} INFO - 读取数据完成.............


In [5]:
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['is_version'] == 0]


In [6]:
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['pay_time'].notnull()]
mid_car_cyp_paimai_order['pay_time'] = mid_car_cyp_paimai_order['pay_time'].map(lambda x:str(x)[:10])

mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['first_license_date'].notnull()]
mid_car_cyp_paimai_order['first_license_date'] = mid_car_cyp_paimai_order['first_license_date'].map(lambda x:str(x)[:10])


In [7]:
mid_car_cyp_paimai_order['mileage_std'] = mid_car_cyp_paimai_order['mileage'].astype('float')
mid_car_cyp_paimai_order['price'] = mid_car_cyp_paimai_order['order_price'].astype('float')

mid_car_cyp_paimai_order['license_month'] = list(map(lambda x,y:date_time_sub(x,y,"%Y-%m"),mid_car_cyp_paimai_order['first_license_date'].map(lambda x:x[:7]),mid_car_cyp_paimai_order['pay_time'].map(lambda x:x[:7])))
mid_car_cyp_paimai_order['license_month'] = mid_car_cyp_paimai_order['license_month'].map(lambda x:round(x/30))


In [8]:
# mid_car_cyp_paimai_order['license_month'] = mid_car_cyp_paimai_order['license_month'].astype('int')

mid_car_cyp_paimai_order['price'] = mid_car_cyp_paimai_order['price']/1000000
mid_car_cyp_paimai_order['guide_price'] = mid_car_cyp_paimai_order['guide_price']/10000
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[~mid_car_cyp_paimai_order['model_name_'].isnull()]

In [9]:
mid_car_cyp_paimai_order['residual'] = mid_car_cyp_paimai_order['price']/mid_car_cyp_paimai_order['guide_price']
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[mid_car_cyp_paimai_order['order_status'] == 80]


In [10]:
# ## 数据清洗，剔除车型匹配错误的数据，找回对车系不同剔除数据过多的数据
mid_car_cyp_paimai_order = mid_car_cyp_paimai_order.loc[(~mid_car_cyp_paimai_order['series_name_'].isnull()) & 
                             (mid_car_cyp_paimai_order['series_name'] == mid_car_cyp_paimai_order['series_name_']) & 
                            (mid_car_cyp_paimai_order['order_status'] == 80)]



In [14]:
car_model_date = '2020-11-02'

In [15]:
# 读取 001 处理好的数据
model = pd.read_csv(curr_dir+car_model_date+"版车型参数及独热编码.csv", header=0, low_memory=False)
# 读取可估价的车型
model = model[['model_code', 'model_year', 'new_car_price','year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6',
                     'year_7', 'year_8', 'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 
                     'year_14', 'year_15', 'year_16','rate','rate_count']]
print("合并前数据数目为:%d, 车型数量为：%d" % (mid_car_cyp_paimai_order.shape[0], len(mid_car_cyp_paimai_order['model_code'].unique())))
mid_car_cyp_paimai_order['model_code'] = mid_car_cyp_paimai_order['model_code'].astype('str')
mid_car_cyp_paimai_order = pd.merge(mid_car_cyp_paimai_order, model, on='model_code', how='inner')
print("<---------------------------------------->")
print("合并后数据数目为:%d, 车型数量为：%d" % (mid_car_cyp_paimai_order.shape[0], len(mid_car_cyp_paimai_order['model_code'].unique())))


合并前数据数目为:253817, 车型数量为：14041
<---------------------------------------->
合并后数据数目为:252985, 车型数量为：13754


In [16]:
mid_car_cyp_paimai_order['license_year'] = mid_car_cyp_paimai_order['first_license_date'].map(lambda x:int(x[:4]))


In [17]:
data_m = mid_car_cyp_paimai_order[(mid_car_cyp_paimai_order['mileage_std']>=0.01) & (mid_car_cyp_paimai_order['mileage_std']<=60)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[(data_m['price']>=0.3) & (data_m['price']<=300)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[(data_m['license_month']>=1) & (data_m['license_month']<=180)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
# data_m = data_m[(data_m['residual']>=0.05) & (data_m['residual']<1)]
# data_m = data_m[(data_m['model_year']>=2005)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[data_m['license_year']>=2005]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))
data_m = data_m[(data_m['new_car_price']>=1) & (data_m['new_car_price']<=500)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))



增加限制后，数据数量为:251950, 车型数量为:13695
增加限制后，数据数量为:245440, 车型数量为:13452
增加限制后，数据数量为:242738, 车型数量为:13306
增加限制后，数据数量为:242738, 车型数量为:13306
增加限制后，数据数量为:235887, 车型数量为:13064
增加限制后，数据数量为:235887, 车型数量为:13064


In [18]:
# 不能参加估计的车型查看 
mid_car_cyp_paimai_order[~mid_car_cyp_paimai_order.model_code.isin(list(set(data_m.model_code)))][['model_year','license_year','license_month','new_car_price','price','mileage_std']].shape


(3511, 6)

In [19]:
#需要计算 meimei 
data_m['per_mile'] = round(data_m['mileage_std']/(data_m['license_month']/12),2)
data_m['year_err'] = data_m['license_year'] - data_m['model_year']
data_m = data_m[(data_m['year_err']>=-2) & (data_m['year_err']<=6)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))


增加限制后，数据数量为:235841, 车型数量为:13052


In [20]:
# 读取编码完成的省份数据
province = pd.read_csv("../Valuation/province_new.csv", header=0, low_memory=False) 
licence = pd.read_excel("licence.xlsx")
licence = licence.rename(columns={"prov":"province_name","city":"city_name"})
if 'province_name' in data_m.columns:
    del data_m['province_name']
if 'city_name' in data_m.columns:
    del data_m['city_name']
    
data_m = pd.merge(data_m,licence[['license','province_name','city_name']],left_on='license_simplify',right_on='license',how='left')
data2m= pd.merge(data_m, province, how='left', on='province_name')

In [23]:
data2m['keep_value'] = data2m[['license_month','year_1', 'year_2', 'year_3', 'year_4','year_5', 'year_6', 'year_7', 'year_8', 'year_9', 'year_10', 'year_11','year_12', 'year_13', 'year_14', 'year_15', 'year_16']].to_dict(orient='records')
data2m['keep_value'] = data2m['keep_value'].map(lambda tar:computer_with_license_month(tar))


In [24]:
data3m = data2m.drop(['year_1', 'year_2', 'year_3', 'year_4', 
                    'year_5', 'year_6', 'year_7', 'year_8', 
                    'year_9', 'year_10', 'year_11', 'year_12', 
                    'year_13', 'year_14', 'year_15', 'year_16'], axis=1)
quality = pd.read_csv(curr_dir+car_model_date+"版车型参数及独热编码.csv", header=0, low_memory=False)
quality = quality[['model_code', 'quality_mile', 'quality_year']]
quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65250 entries, 0 to 65249
Data columns (total 3 columns):
model_code      65250 non-null object
quality_mile    65250 non-null float64
quality_year    65250 non-null float64
dtypes: float64(2), object(1)
memory usage: 1.5+ MB


In [25]:
data3m = pd.merge(data3m, quality, how='left', on='model_code')
def get_quality(license_month,mileage_std,quality_mile,quality_year):
    year = round(license_month/12, 2)
    if mileage_std < quality_mile and year< quality_year:
        return 1
    else:
        return 0
       
data3m['quality'] = list(map(lambda license_month,mileage_std,quality_mile,quality_year:get_quality(license_month,mileage_std,quality_mile,quality_year),
          data3m['license_month'],data3m['mileage_std'],data3m['quality_mile'],data3m['quality_year']))
data3m = data3m.drop(['quality_mile', 'quality_year'], axis=1)


In [26]:
#计算一下log  替换成 hive 计算  代码更新
data3m['mile_log']=round(np.log2(data3m['mileage_std']), 4)
data3m['license_month_log'] = data3m['license_month'].map(lambda x:round(math.log(x,100),2))
data3m['new_car_price_log']=round(np.log2(data3m['new_car_price']), 4)


In [27]:
car = pd.read_csv(curr_dir+car_model_date+"版车型参数及独热编码.csv", header = 0, low_memory=False)
car=car.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)


In [28]:
data3m = pd.merge(data3m, car, how="inner", on="model_code")
vec = pd.read_csv(curr_dir+car_model_date+"版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
vec = vec.drop(['model_name'], axis=1)
data3m = pd.merge(data3m, vec, how="inner", on="model_code")
data3m['pay_time'] = data3m['pay_time'].map(lambda x:x[:10])
data3m['days'] = list(map(lambda x:date_time_sub(x,'2020-12-31',"%Y-%m-%d"),data3m['pay_time']))


In [29]:
import math
data3m['days_log'] = data3m['days'].map(lambda x:round(math.log(x,100),1))
data3m['month'] = data3m['days'].map(lambda x:round(x/30))


In [34]:
## 增加车系、品牌编码
brand_code_dict = {}
for i,brand_code in enumerate(data3m['brand_code'].unique()):
    brand_code_dict[brand_code] = i 
data3m['brand_code_index'] = data3m['brand_code'].map(lambda x:brand_code_dict[x])

series_code_dict = {}
for i,series_code in enumerate(data3m['series_code'].unique()):
    series_code_dict[series_code] = i 
data3m['series_code_index'] = data3m['series_code'].map(lambda x:series_code_dict[x])

province_name_dict = {}
for i,province_name in enumerate(data3m['province_name'].unique()):
    province_name_dict[province_name] = i 
data3m['province_name_index'] = data3m['province_name'].map(lambda x:province_name_dict[x])


In [35]:
fearture_columns = ['month', 'license_year', 'model_year', 'rate', 'rate_count', 'per_mile', 'year_err',
                    'brand_code_index','series_code_index','province_name_index',
                    'keep_value', 'quality', 'mile_log', 'license_month_log', 'new_car_price_log',
                    'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'engine_volume_l', 'cylinder_number', 
                    'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 
                    'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 
                    'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 
                    'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'country_id0', 'country_id1', 
                    'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 
                    'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 
                    'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 
                    'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 
                    'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3',
                    'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 
                    'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 
                    'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 
                    'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 
                    'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 
                    'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29',
                    'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35',
                    'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 
                    'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 
                    'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 
                    'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 
                    'series_level60', 'series_level61', 'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 
                    'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14',
                    'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23',
                    'wordvec24', 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 
                    'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41',
                    'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50',
                    'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 
                    'wordvec60', 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 
                    'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77',
                    'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86',
                    'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 
                    'wordvec96', 'wordvec97', 'wordvec98', 'wordvec99']

label = 'residual'


In [37]:
## rank_desc,rank_level,color_name,tradetimes,'purpose','isblister'
## purpose 车辆使用性质（10非营运、20 营运  30  营转非  40  租赁  50  教练...
## isblister 是否水泡车辆  0 否  1 是  默认 0
## isaccident 是否事故车辆 0:非事故车 1:事故车
# data3m.loc[data3m['purpose'] == 10,'purpose'] = '10'
# data3m = data3m.loc[data3m['purpose'].isin(['10', '20', '30', '40', '50'])]
## 产品模式，竞拍玩法 场次产品线：1-现场拍、2-快拍、4-易拍、5-线上撮合、72-面对面交易、77-暗拍、
##1001-车道拍、1011-超级拍（现场拍）、1021-超级拍(车道拍)、 25-现场拍、35-线下交易、36-公司收购、7-原公司收购、
##73-卖家直拍、74-随时拍、76-自行交易、1002-批售暗拍

data3m.loc[data3m['use_type'] == 1.0,'purpose'] = '10'
data3m.loc[data3m['use_type'] == 2.0,'purpose'] = '30'  
data3m.loc[data3m['use_type'] == 0.0,'purpose'] = '20'
data3m.loc[data3m['use_type'] == 5.0,'purpose'] = '40'
data3m.loc[data3m['use_type'] == 6.0,'purpose'] = '50'
# data3m.loc[data3m['purpose'] == 10,'purpose'] = '10'
data3m = data3m.loc[data3m['purpose'].isin(['10', '20', '30', '40', '50'])]
print("使用性质",data3m.shape)
data3m = data3m.loc[data3m['rank_desc'].isin(['A','B','C','D'])]

data3m['tradetimes'] = data3m['tradetimes'].fillna(-1)
data3m.loc[data3m['tradetimes']>10,'tradetimes'] = 10
data3m['tradetimes'] = data3m['tradetimes'].map(lambda x:str(int(x)))

data3m['rank_level'] = data3m['rank_level'].map(lambda x:str(x))

data3m['color_name'] = data3m['color_name'].map(lambda x:str(x).replace('色','').replace('/','').replace('／',''))
data3m.loc[data3m['color_name'].isin(['nan', '不详','其他颜']),'color_name'] = '其他'
data3m.loc[~data3m['color_name'].isin(['银','粉','绿','紫','棕','黄','蓝','红','白','黑','灰','其他']),
           'color_name'] = '多色'

"""
data3m['color_name'] = data3m['color_name'].map(lambda x:str(x).replace('色','').replace('/','').replace('／','')\
.replace('绿黑','黑绿').replace('白黑','黑白').replace('红黑','黑红').replace('黑黄','黄黑').replace('白绿','绿白').\
replace('紫黑','黑紫').replace('灰黑','黑灰').replace('白蓝','蓝白').replace('红白','白红').\
                         replace('棕白','白棕').replace('黑蓝','蓝黑').replace('灰综','灰棕').\
                         replace('棕灰','灰棕').replace('棕黑','黑棕').replace('灰黄','黄灰').
                        replace('蓝灰','灰蓝').replace('蓝黄','黄蓝').replace('灰红','红灰'))

data3m.loc[data3m['color_name'].isin(['其他颜','黑白红','nan','白黑红','蓝黑白','黑红灰','黑灰白',' 墨玉黑','蓝白红','黑白黄']),'color_name'] = '其他'
"""
data3m = data3m.loc[~data3m['auction_mode'].isnull()]
print("剔除数据",data3m.shape)
data3m['auction_mode'] = data3m['auction_mode'].map(lambda x:str(int(x)))


使用性质 (235443, 355)
剔除数据 (232792, 355)


In [38]:
data3m = data3m.loc[data3m['auction_mode'].isin(['2','4','1','1001','1011','1021','74'])]
print("auction_mode剔除数据",data3m.shape)


auction_mode剔除数据 (182598, 355)


In [39]:
data3m.loc[data3m['environmental_standards'].isnull(),'environmental_standards'] = '14'


In [40]:
## 增加排放标准和排放标准数据处理
environmental_standards_dict = {"0":"国II","1":"国IV","2":"国III","3":"国IV","4":"国III","5":"国III","6":"国II",
"7":"国IV","8":"国IV","9":"国V","10":"国IV","11":"国III","12":"国V","13":"国V",
"14":"未知","15":"国VI","16":"国IV","17":"国V","19":"国V","20":"国V","21":"国V","22":"国I",
"23":"国IV","24":"国IV","25":"国IV","26":"国V","27":"国III","28":"国V","30":"国VI",
"31":"国VI","32":"国VI","33":"未知","34":"化油器","35":"纯电动","36":"国V","37":"国VI",
"38":"国IV","39":"国III","40":"国V","41":"国III","42":"国IV"}

environmental_standards_odb = ["5","7","8","11","12","16","21","23","24","26","27","32","41"]
data3m['environmental'] = data3m['environmental_standards'].map(lambda x:environmental_standards_dict[str(int(x))])

data3m.loc[data3m['environmental_standards'].map(lambda x:str(int(x))).isin(environmental_standards_odb),'OBD'] = 1 
data3m['OBD'] = data3m['OBD'].fillna(0)

In [46]:
## 精准估值信息编码
rank_desc_dict = {}
for i,rank_desc in enumerate(data3m['rank_desc'].unique()):
    rank_desc_dict[rank_desc] = i 
data3m['rank_desc_index'] = data3m['rank_desc'].map(lambda x:rank_desc_dict[x])

rank_level_dict = {}
for i,rank_level in enumerate(data3m['rank_level'].unique()):
    rank_level_dict[rank_level] = i 
data3m['rank_level_index'] = data3m['rank_level'].map(lambda x:rank_level_dict[x])

tradetimes_dict = {}
for i,tradetimes in enumerate(data3m['tradetimes'].unique()):
    tradetimes_dict[tradetimes] = i 
data3m['tradetimes_index'] = data3m['tradetimes'].map(lambda x:tradetimes_dict[x])

color_name_dict = {}
for i,color_name in enumerate(data3m['color_name'].unique()):
    color_name_dict[color_name] = i 
data3m['color_name_index'] = data3m['color_name'].map(lambda x:color_name_dict[x])

purpose_dict = {}
for i,purpose in enumerate(data3m['purpose'].unique()):
    purpose_dict[purpose] = i 
data3m['purpose_index'] = data3m['purpose'].map(lambda x:purpose_dict[x])


In [47]:
fearture_columns_2 = ['rank_desc_index','rank_level_index','tradetimes_index','color_name_index','purpose_index']
label = 'residual'

In [48]:
data3m['per_mile_log'] = data3m['per_mile'].map(lambda x:math.log(round(x,1)+1,2))
data3m['keep_value_log'] = data3m['keep_value'].map(lambda x:math.log(x))


In [49]:

data3m['license_year'] = data3m['license_month'].map(lambda x:x//30)

In [50]:
data3m['car_years_year'] = data3m['license_month'].map(lambda x:x//12)
data3m['car_years_month'] = data3m['license_month'].map(lambda x:x%12)


In [52]:
fearture_columns = ['month', 'license_year', 'model_year','license_month_log', 'rate', 'rate_count', 'per_mile_log', 'year_err',
                    'brand_code_index','series_code_index','province_name_index',
                    'keep_value_log', 'quality', 'new_car_price_log', 
                    'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'cylinder_number', 
                    'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 
                    'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 
                    'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 
                    'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'country_id0', 'country_id1', 
                    'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 
                    'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 
                    'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 
                    'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 
                    'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3',
                    'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 
                    'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 
                    'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 
                    'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 
                    'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 
                    'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29',
                    'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35',
                    'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 
                    'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 
                    'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 
                    'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 
                    'series_level60', 'series_level61', 'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 
                    'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14',
                    'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23',
                    'wordvec24', 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 
                    'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41',
                    'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50',
                    'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 
                    'wordvec60', 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 
                    'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77',
                    'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86',
                    'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 
                    'wordvec96', 'wordvec97', 'wordvec98', 'wordvec99']


label = 'residual'


In [53]:
logger.log("处理基础特征数据完成.............")

[2020-12-01 15:04:36,139] {<ipython-input-2-18214e15c7d3>:85} INFO - 处理基础特征数据完成.............


# 常规方式训练模型

In [56]:
# data3m[['action_mode','model_code']].groupby('action_mode').count().reset_index()
data3m = data3m.loc[~data3m[label].isnull()]
data3m = data3m.loc[data3m['order_status'] == 80]
data3m = data3m.loc[data3m[label]<0.99]
data3m['model_year'] = data3m['model_year'].map(lambda x:int(x))
data3m = data3m.loc[~data3m['province_name'].isnull()]

test_data = data3m.loc[(data3m['pay_time']>='2020-11-01') & 
                       (data3m['pay_time']<'2020-12-01') & 
                       (data3m['tenant_name'] != '弹个车拍卖店铺') & 
                        (data3m['seller_name'] != '弹个车线上测试店铺')& 
                        (data3m['auction_mode'].isin(['2','4'])) ]

train_data = data3m.loc[(data3m['pay_time']<'2020-11-01') & 
                        (data3m['pay_time']>'2016-01-01') & 
                        (data3m['tenant_name'] != '弹个车拍卖店铺') & 
                        (data3m['seller_name'] != '弹个车线上测试店铺') ]



In [57]:
print(data3m.shape,train_data.shape,test_data.shape)

(182335, 366) (166061, 366) (2772, 366)


In [58]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 5000,
                        max_depth = 11,
                        num_leaves = 100, 
                        subsample = 1, 
                        colsample_bytree = 0.34,
                        min_child_samples = 110, 
                        n_jobs = 8,
                        verbose = 1)
# gbm.fit(train_data[fearture_columns+fearture_columns_2], train_data[label])

gbm.fit(train_data[fearture_columns+fearture_columns_2], train_data[label],
        eval_set = (test_data[fearture_columns+fearture_columns_2], test_data[label]),
        eval_metric = {'l1','l2'}, ,
        categorical_feature=['province_name','model_year','license_time_year', 'publish_time_year','quality',
                            'brand_code_index','series_code_index']
        early_stopping_rounds = 50, 
        verbose=True)


[1]	valid_0's l2: 0.0272143	valid_0's l1: 0.128882
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0258228	valid_0's l1: 0.125356
[3]	valid_0's l2: 0.0245461	valid_0's l1: 0.122069
[4]	valid_0's l2: 0.0232904	valid_0's l1: 0.118796
[5]	valid_0's l2: 0.0225827	valid_0's l1: 0.116848
[6]	valid_0's l2: 0.0214122	valid_0's l1: 0.113611
[7]	valid_0's l2: 0.0207736	valid_0's l1: 0.111794
[8]	valid_0's l2: 0.0197803	valid_0's l1: 0.108927
[9]	valid_0's l2: 0.0188104	valid_0's l1: 0.106079
[10]	valid_0's l2: 0.0178657	valid_0's l1: 0.103289
[11]	valid_0's l2: 0.0173338	valid_0's l1: 0.101717
[12]	valid_0's l2: 0.0164845	valid_0's l1: 0.0990707
[13]	valid_0's l2: 0.0160082	valid_0's l1: 0.0975633
[14]	valid_0's l2: 0.0152207	valid_0's l1: 0.0950145
[15]	valid_0's l2: 0.0148047	valid_0's l1: 0.0935278
[16]	valid_0's l2: 0.0144241	valid_0's l1: 0.0922515
[17]	valid_0's l2: 0.0137326	valid_0's l1: 0.0898858
[18]	valid_0's l2: 0.0131316	valid_0's l1: 0.0879424
[19]

[152]	valid_0's l2: 0.00167841	valid_0's l1: 0.0293415
[153]	valid_0's l2: 0.00167303	valid_0's l1: 0.0292908
[154]	valid_0's l2: 0.00166867	valid_0's l1: 0.029251
[155]	valid_0's l2: 0.0016656	valid_0's l1: 0.0292151
[156]	valid_0's l2: 0.00166251	valid_0's l1: 0.0291837
[157]	valid_0's l2: 0.0016579	valid_0's l1: 0.0291474
[158]	valid_0's l2: 0.00165526	valid_0's l1: 0.0291235
[159]	valid_0's l2: 0.00164948	valid_0's l1: 0.0290647
[160]	valid_0's l2: 0.00164628	valid_0's l1: 0.0290355
[161]	valid_0's l2: 0.0016406	valid_0's l1: 0.0289751
[162]	valid_0's l2: 0.00163742	valid_0's l1: 0.0289398
[163]	valid_0's l2: 0.0016341	valid_0's l1: 0.0289028
[164]	valid_0's l2: 0.00163143	valid_0's l1: 0.0288764
[165]	valid_0's l2: 0.00162728	valid_0's l1: 0.0288359
[166]	valid_0's l2: 0.00162306	valid_0's l1: 0.0287964
[167]	valid_0's l2: 0.00161997	valid_0's l1: 0.0287694
[168]	valid_0's l2: 0.00161704	valid_0's l1: 0.0287414
[169]	valid_0's l2: 0.00161354	valid_0's l1: 0.0286994
[170]	valid_0's

[302]	valid_0's l2: 0.00129158	valid_0's l1: 0.0255436
[303]	valid_0's l2: 0.00129054	valid_0's l1: 0.0255339
[304]	valid_0's l2: 0.00128928	valid_0's l1: 0.0255237
[305]	valid_0's l2: 0.001287	valid_0's l1: 0.0255037
[306]	valid_0's l2: 0.00128523	valid_0's l1: 0.0254876
[307]	valid_0's l2: 0.0012834	valid_0's l1: 0.025468
[308]	valid_0's l2: 0.00128221	valid_0's l1: 0.0254552
[309]	valid_0's l2: 0.00128073	valid_0's l1: 0.0254391
[310]	valid_0's l2: 0.00127985	valid_0's l1: 0.0254302
[311]	valid_0's l2: 0.00127788	valid_0's l1: 0.0254077
[312]	valid_0's l2: 0.00127662	valid_0's l1: 0.0253957
[313]	valid_0's l2: 0.00127456	valid_0's l1: 0.0253755
[314]	valid_0's l2: 0.00127403	valid_0's l1: 0.025369
[315]	valid_0's l2: 0.00127345	valid_0's l1: 0.0253631
[316]	valid_0's l2: 0.00127253	valid_0's l1: 0.0253519
[317]	valid_0's l2: 0.00127126	valid_0's l1: 0.025337
[318]	valid_0's l2: 0.00127009	valid_0's l1: 0.0253225
[319]	valid_0's l2: 0.00126804	valid_0's l1: 0.0253037
[320]	valid_0's 

[452]	valid_0's l2: 0.00113587	valid_0's l1: 0.0238858
[453]	valid_0's l2: 0.00113491	valid_0's l1: 0.023875
[454]	valid_0's l2: 0.00113365	valid_0's l1: 0.0238609
[455]	valid_0's l2: 0.00113312	valid_0's l1: 0.0238563
[456]	valid_0's l2: 0.00113276	valid_0's l1: 0.0238488
[457]	valid_0's l2: 0.00113186	valid_0's l1: 0.0238431
[458]	valid_0's l2: 0.00113137	valid_0's l1: 0.0238375
[459]	valid_0's l2: 0.00113138	valid_0's l1: 0.0238363
[460]	valid_0's l2: 0.00113062	valid_0's l1: 0.0238294
[461]	valid_0's l2: 0.00113043	valid_0's l1: 0.0238236
[462]	valid_0's l2: 0.00113003	valid_0's l1: 0.0238161
[463]	valid_0's l2: 0.00112965	valid_0's l1: 0.023812
[464]	valid_0's l2: 0.00112913	valid_0's l1: 0.0238089
[465]	valid_0's l2: 0.00112882	valid_0's l1: 0.0238058
[466]	valid_0's l2: 0.00112813	valid_0's l1: 0.0237983
[467]	valid_0's l2: 0.00112715	valid_0's l1: 0.0237862
[468]	valid_0's l2: 0.00112671	valid_0's l1: 0.023781
[469]	valid_0's l2: 0.00112623	valid_0's l1: 0.0237769
[470]	valid_0

[602]	valid_0's l2: 0.00104872	valid_0's l1: 0.0229542
[603]	valid_0's l2: 0.00104775	valid_0's l1: 0.0229393
[604]	valid_0's l2: 0.00104763	valid_0's l1: 0.0229394
[605]	valid_0's l2: 0.00104689	valid_0's l1: 0.022931
[606]	valid_0's l2: 0.00104671	valid_0's l1: 0.0229285
[607]	valid_0's l2: 0.00104617	valid_0's l1: 0.0229216
[608]	valid_0's l2: 0.0010457	valid_0's l1: 0.0229177
[609]	valid_0's l2: 0.00104522	valid_0's l1: 0.0229127
[610]	valid_0's l2: 0.00104475	valid_0's l1: 0.0229077
[611]	valid_0's l2: 0.00104409	valid_0's l1: 0.0229002
[612]	valid_0's l2: 0.00104366	valid_0's l1: 0.0228955
[613]	valid_0's l2: 0.00104324	valid_0's l1: 0.0228912
[614]	valid_0's l2: 0.00104261	valid_0's l1: 0.0228856
[615]	valid_0's l2: 0.00104202	valid_0's l1: 0.0228791
[616]	valid_0's l2: 0.00104131	valid_0's l1: 0.0228729
[617]	valid_0's l2: 0.00104065	valid_0's l1: 0.0228654
[618]	valid_0's l2: 0.00104025	valid_0's l1: 0.0228598
[619]	valid_0's l2: 0.00104004	valid_0's l1: 0.0228583
[620]	valid_

...

[2206]	valid_0's l2: 0.000870893	valid_0's l1: 0.0206419
[2207]	valid_0's l2: 0.00087091	valid_0's l1: 0.0206428
[2208]	valid_0's l2: 0.000870891	valid_0's l1: 0.020641
[2209]	valid_0's l2: 0.000870893	valid_0's l1: 0.0206404
[2210]	valid_0's l2: 0.000870897	valid_0's l1: 0.0206404
[2211]	valid_0's l2: 0.00087087	valid_0's l1: 0.0206392
[2212]	valid_0's l2: 0.000870769	valid_0's l1: 0.020638
[2213]	valid_0's l2: 0.000870809	valid_0's l1: 0.0206379
[2214]	valid_0's l2: 0.000870837	valid_0's l1: 0.0206389
[2215]	valid_0's l2: 0.000870758	valid_0's l1: 0.0206396
[2216]	valid_0's l2: 0.000870681	valid_0's l1: 0.020639
[2217]	valid_0's l2: 0.000870658	valid_0's l1: 0.0206395
[2218]	valid_0's l2: 0.000870579	valid_0's l1: 0.0206401
[2219]	valid_0's l2: 0.000870636	valid_0's l1: 0.0206416
[2220]	valid_0's l2: 0.00087059	valid_0's l1: 0.0206407
[2221]	valid_0's l2: 0.000870564	valid_0's l1: 0.0206396
[2222]	valid_0's l2: 0.000870111	valid_0's l1: 0.0206343
[2223]	valid_0's l2: 0.000870094	vali

[2351]	valid_0's l2: 0.00086522	valid_0's l1: 0.0205664
[2352]	valid_0's l2: 0.000865222	valid_0's l1: 0.0205664
[2353]	valid_0's l2: 0.000865241	valid_0's l1: 0.0205665
[2354]	valid_0's l2: 0.00086528	valid_0's l1: 0.0205667
[2355]	valid_0's l2: 0.000865354	valid_0's l1: 0.0205677
[2356]	valid_0's l2: 0.000865368	valid_0's l1: 0.0205678
[2357]	valid_0's l2: 0.000865326	valid_0's l1: 0.0205673
[2358]	valid_0's l2: 0.000865217	valid_0's l1: 0.0205655
[2359]	valid_0's l2: 0.00086519	valid_0's l1: 0.0205644
[2360]	valid_0's l2: 0.000865106	valid_0's l1: 0.0205627
[2361]	valid_0's l2: 0.000865159	valid_0's l1: 0.0205623
[2362]	valid_0's l2: 0.00086522	valid_0's l1: 0.0205616
[2363]	valid_0's l2: 0.000865176	valid_0's l1: 0.0205615
[2364]	valid_0's l2: 0.00086528	valid_0's l1: 0.0205625
[2365]	valid_0's l2: 0.000865239	valid_0's l1: 0.0205621
[2366]	valid_0's l2: 0.000865271	valid_0's l1: 0.0205625
[2367]	valid_0's l2: 0.000865173	valid_0's l1: 0.020561
[2368]	valid_0's l2: 0.000865212	vali

[2496]	valid_0's l2: 0.000862906	valid_0's l1: 0.0205239
[2497]	valid_0's l2: 0.000862904	valid_0's l1: 0.0205239
[2498]	valid_0's l2: 0.000862826	valid_0's l1: 0.0205226
[2499]	valid_0's l2: 0.000862792	valid_0's l1: 0.0205226
[2500]	valid_0's l2: 0.000862749	valid_0's l1: 0.0205215
[2501]	valid_0's l2: 0.000862868	valid_0's l1: 0.0205226
[2502]	valid_0's l2: 0.000862874	valid_0's l1: 0.0205224
[2503]	valid_0's l2: 0.000862707	valid_0's l1: 0.0205204
[2504]	valid_0's l2: 0.0008629	valid_0's l1: 0.020521
[2505]	valid_0's l2: 0.000862783	valid_0's l1: 0.0205192
[2506]	valid_0's l2: 0.000862833	valid_0's l1: 0.0205195
[2507]	valid_0's l2: 0.000862822	valid_0's l1: 0.020519
[2508]	valid_0's l2: 0.000862892	valid_0's l1: 0.0205204
[2509]	valid_0's l2: 0.000862709	valid_0's l1: 0.020519
[2510]	valid_0's l2: 0.000862712	valid_0's l1: 0.0205178
[2511]	valid_0's l2: 0.000862769	valid_0's l1: 0.0205176
[2512]	valid_0's l2: 0.000862711	valid_0's l1: 0.0205173
[2513]	valid_0's l2: 0.000862789	val

[2641]	valid_0's l2: 0.00086144	valid_0's l1: 0.0204882
[2642]	valid_0's l2: 0.000861479	valid_0's l1: 0.0204885
[2643]	valid_0's l2: 0.000861518	valid_0's l1: 0.0204879
[2644]	valid_0's l2: 0.000861499	valid_0's l1: 0.0204877
[2645]	valid_0's l2: 0.000861506	valid_0's l1: 0.0204881
[2646]	valid_0's l2: 0.000861535	valid_0's l1: 0.0204881
[2647]	valid_0's l2: 0.000861509	valid_0's l1: 0.0204878
[2648]	valid_0's l2: 0.000861506	valid_0's l1: 0.0204878
[2649]	valid_0's l2: 0.000861509	valid_0's l1: 0.0204878
[2650]	valid_0's l2: 0.000861524	valid_0's l1: 0.0204875
[2651]	valid_0's l2: 0.000861632	valid_0's l1: 0.0204885
[2652]	valid_0's l2: 0.000861636	valid_0's l1: 0.0204883
[2653]	valid_0's l2: 0.000861615	valid_0's l1: 0.020488
[2654]	valid_0's l2: 0.000861577	valid_0's l1: 0.020488
[2655]	valid_0's l2: 0.000861492	valid_0's l1: 0.0204872
[2656]	valid_0's l2: 0.00086144	valid_0's l1: 0.0204869
[2657]	valid_0's l2: 0.000861396	valid_0's l1: 0.0204857
[2658]	valid_0's l2: 0.0008614	vali

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.34,
       learning_rate=0.03, max_depth=11, min_child_samples=110,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=5000,
       n_jobs=8, num_leaves=100, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [59]:
feature_importances_df = pd.DataFrame(fearture_columns+fearture_columns_2,columns=['fearture'])
feature_importances_df['feature_importances'] = gbm.feature_importances_

# feature_importances_df.loc[feature_importances_df['feature_importances']>0].sort_values(by='feature_importances',ascending=False)
# feature_importances_df = pd.merge(feature_importances_df,check_item_value[['check_item_value_code','check_item_value_name']],left_on='fearture',right_on='check_item_value_code',how='left')
feature_importances_df.sort_values(by='feature_importances',ascending=False)

,fearture,feature_importances
0,month,10569
11,keep_value_log,7874
3,license_month_log,7605
6,per_mile_log,5693
2,model_year,5101
13,new_car_price_log,5015
10,province_name_index,4357
5,rate_count,3327
254,color_name_index,2875
15,length,2813


In [61]:
from sklearn.metrics import r2_score
test_data['predict_0'] = gbm.predict(test_data[fearture_columns+fearture_columns_2].fillna(0))
test_t2 = r2_score(test_data['residual'],test_data['predict_0'])
train_data['predict_0'] = gbm.predict(train_data[fearture_columns+fearture_columns_2].fillna(0))
train_t2 = r2_score(train_data['residual'],train_data['predict_0'])
print(train_t2,test_t2)#,veri_t2)


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.9837864592749612 0.9687157985028845


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [62]:
test_data['差异'] = (test_data['predict_0']-test_data['residual'])/test_data['residual']
train_data['差异'] = (train_data['predict_0']-train_data['residual'])/train_data['residual']


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
## 数据结果评估
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))

print("测试估值。。。")
logger.log("精准估值模型 测试数据评估.............")
data3m_pinggu(test_data,'差异')
print("训练估值。。。")
logger.log("精准估值模型 训练数据评估.............")
data3m_pinggu(train_data,'差异')


测试估值。。。
[2020-12-01 16:28:21,393] {<ipython-input-2-18214e15c7d3>:85} INFO - 精准估值模型 测试数据评估.............
P<3%:  0.2825
P<5%:  0.4499
P<8%:  0.6219
P<10%:  0.7154
P<20%:  0.9206
训练估值。。。
[2020-12-01 16:28:21,422] {<ipython-input-2-18214e15c7d3>:85} INFO - 精准估值模型 训练数据评估.............
P<3%:  0.3681


In [59]:
data3m_pinggu(test_data.loc[test_data['environmental'] != '国III'],'差异')


P<3%:  0.2847
P<5%:  0.4523
P<8%:  0.6341
P<10%:  0.7286
P<20%:  0.9233


In [456]:
"""
    计算置信度数据表
"""
train_data['predict_0'] = gbm.predict(train_data[fearture_columns+fearture_columns_2].fillna(0))
# data3m = pd.merge(data3m,data3m_adjust,on='model_code',how='left')
train_data['predict_1'] = train_data['predict_0'] #- data3m['adjust_residual']

train_data['差异'] = (train_data['predict_1']-train_data['residual'])/train_data['residual']

data3m_err_count = train_data.loc[(train_data['差异']> 0.105)][['model_code','order_id']].groupby('model_code').count().\
                reset_index().rename(columns={"order_id":"err_count"})

data3m_count = train_data[['model_code','order_id']].groupby('model_code').count().\
                reset_index().rename(columns={"order_id":"count"})

data3m_count = pd.merge(data3m_count,data3m_err_count,on='model_code',how='left')
data3m_count['置信度'] = (data3m_count['count'] - data3m_count['err_count'].fillna(0))/ data3m_count['count'] 

## 到处测试数据
file_name = "车易拍精准估值置信度.xlsx"
data3m_count.fillna(0).to_excel(curr_dir+file_name)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation_cyp/"+file_name, curr_dir+file_name)


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

# 车况明细的模型

In [67]:
"""
    增加检测明细的数据
"""
import pandas as pd 
data_df_list = []
for i in range(50):
    threadName = 'Thread_0_2019_'+str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)
    
for i in range(30):
    threadName = 'Thread_0_2018_'+str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)
    
for i in range(30):
    threadName = 'Thread_0_2017_'+str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)
    
for i in range(30):
    threadName = 'Thread_0_2016_'+str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)

## 截止到2020-08-26之前2020年的数据
data_df = pd.read_excel(curr_dir+'/检测报告数据_2020.xlsx')
data_df_list.append(data_df)

## 2020-08-26~2020-09-26时间段的数据
for i in range(0,5):
    threadName = 'Thread_0_20200826_20200926_'+str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)


Thread_0_2019_0 (1626, 95) (65, 95)
Thread_0_2019_1 (1626, 95) (37, 95)
Thread_0_2019_2 (1626, 95) (44, 95)
Thread_0_2019_3 (1626, 95) (36, 95)
Thread_0_2019_4 (1626, 95) (45, 95)
Thread_0_2019_5 (1626, 95) (37, 95)
Thread_0_2019_6 (1626, 95) (43, 95)
Thread_0_2019_7 (1626, 95) (38, 95)
Thread_0_2019_8 (1626, 95) (37, 95)
Thread_0_2019_9 (1626, 95) (29, 95)
Thread_0_2019_10 (1626, 95) (42, 95)
Thread_0_2019_11 (1626, 95) (35, 95)
Thread_0_2019_12 (1626, 95) (42, 95)
Thread_0_2019_13 (1626, 95) (39, 95)
Thread_0_2019_14 (1626, 95) (44, 95)
Thread_0_2019_15 (1626, 95) (41, 95)
Thread_0_2019_16 (1626, 95) (37, 95)
Thread_0_2019_17 (1626, 95) (34, 95)
Thread_0_2019_18 (1626, 95) (34, 95)
Thread_0_2019_19 (1626, 95) (34, 95)
Thread_0_2019_20 (1626, 95) (40, 95)
Thread_0_2019_21 (1626, 95) (45, 95)
Thread_0_2019_22 (1626, 95) (45, 95)
Thread_0_2019_23 (1626, 95) (29, 95)
Thread_0_2019_24 (1626, 95) (29, 95)
Thread_0_2019_25 (1626, 95) (33, 95)
Thread_0_2019_26 (1626, 95) (41, 95)
Thread_0_20

In [68]:
# data_df_list_n = []
for i in range(5):
    threadName = 'Thread_0_check_data_null_' + str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)

for i in range(5):
    threadName = 'Thread_0_check_data_null_1_' + str(i)
    data_df = pd.read_excel(curr_dir+'/data_df_df_gujia_'+threadName+'.xlsx')
    data_df_list.append(data_df)
    print(threadName,data_df.shape,data_df.loc[data_df['检测明细描述'].isnull()].shape)


Thread_0_check_data_null_0 (3645, 86) (2, 86)
Thread_0_check_data_null_1 (3645, 88) (2, 88)
Thread_0_check_data_null_2 (3645, 88) (5, 88)
Thread_0_check_data_null_3 (3645, 88) (7, 88)
Thread_0_check_data_null_4 (3645, 95) (175, 95)
Thread_0_check_data_null_1_0 (617, 88) (10, 88)
Thread_0_check_data_null_1_1 (617, 88) (8, 88)
Thread_0_check_data_null_1_2 (617, 95) (23, 95)
Thread_0_check_data_null_1_3 (617, 95) (77, 95)
Thread_0_check_data_null_1_4 (617, 95) (89, 95)


In [69]:
## 合并数据
data_df = pd.concat(data_df_list,axis=0)
data_df= data_df.drop_duplicates()
print('数据量：',data_df.shape)
print('列名称：',data_df.columns)
data_df = data_df.loc[~data_df['检测明细描述'].isnull()]


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


数据量： (283994, 95)
列名称： Index(['ABS', 'batteryCapacity', 'batteryVoltage', 'bodyColor', 'carLevel',
       'carRegDate', 'carVisuals', 'commercialDate', 'drivingMotorPower',
       'emissionStandard', 'factoryDate', 'fined', 'fuelType', 'gearboxForm',
       'inspectionDate', 'insuranceAddress', 'insureDate', 'isBusCheckList',
       'isCarTicket', 'isCheckVoucher', 'isCnameplate', 'isDeclaration',
       'isImport', 'isInvoiceSeal', 'isLicenseTag', 'isStandbyKey',
       'isSurtaxCard', 'isTransferTicket', 'lastAddress', 'lastTradeDate',
       'licenseAddress', 'licenseModel', 'manufacturerName', 'mileage',
       'mileageCount', 'ministryMileage', 'nowUsed', 'oldUsed', 'purpose',
       'rank', 'rankDesc', 'regAddress', 'score', 'scoreDesc', 'seats',
       'sweptVolume', 'tradeTimes', 'trade_code', 'wheelsSize', '一键启动',
       '主驾座椅功能', '主驾座椅材质', '主驾座椅调节方式', '买方负责', '事故检查', '侧滑门打开方式', '倒车影像',
       '倒车雷达', '其他检查', '副驾座椅功能', '副驾座椅调节方式', '动力检查', '卖方责任', '后备箱开启方式',
       '后排座椅功能', '启

In [71]:
data_df['买方负责_len'] = data_df['买方负责'].map(lambda x:len(str(x)))
trade_code_len = data_df[['trade_code','买方负责_len']].groupby('trade_code').max().reset_index().rename(columns={"买方负责_len":"max_len"})

data_df = pd.merge(data_df,trade_code_len,on='trade_code',how='left')
data_df = data_df.loc[data_df['买方负责_len'] == data_df['max_len']]

data_df['trade_code_1'] = data_df['trade_code']
trade_code_count = data_df[['trade_code','trade_code_1']].groupby('trade_code').count().reset_index()

In [72]:
## 获取需要产生的列名称
key_code_value = []
"""
def check_items(items_value):
    items_value_dict = {}
    if str(items_value) != 'nan':
        for k in items_value.split(";"):
            k = k.replace('  ',' ')
            key_code = k.split(" ")[0]
            for i in  k.split(" ")[1].split(','):
                if key_code+"_"+i not in key_code_value: ## key_code+"_"+i
                    key_code_value.append(key_code+"_"+i)
                    ##key_code_value.append(key_code+"_"+i)
            if key_code in items_value_dict.keys():
                items_value_dict[key_code] = list(set(items_value_dict[key_code] +k.split(" ")[1].split(',')))
            else:
                items_value_dict[key_code] = k.split(" ")[1].split(',')
    return items_value_dict
"""

def check_items(items_value):
    items_value_dict = {}
    if str(items_value) != 'nan':
        for k in items_value.split(";"):
            k = k.replace('  ',' ')
            key_code = k.split(" ")[0]
            for i in [i.replace('\n','') for i in k.split(" ")[1].split(',')] : #k.split(" ")[1].split(','):
                if key_code+"_"+i not in key_code_value: ## key_code+"_"+i
                    key_code_value.append(key_code+"_"+i)
                    ##key_code_value.append(key_code+"_"+i)
            if key_code in items_value_dict.keys():
                items_value_dict[key_code] = list(set(items_value_dict[key_code] +[i.replace('\n','') for i in k.split(" ")[1].split(',')]))
            else:
                items_value_dict[key_code] = [i.replace('\n','') for i in k.split(" ")[1].split(',')]
    return items_value_dict

data_df['检测明细描述_d'] = data_df['检测明细描述'].map(lambda x:check_items(x))


In [74]:
# ## 把处理数据为key value的形式
def check_item_code_fun(x,check_item_code):
    code = check_item_code.split("_")[0]
    value = check_item_code.split("_")[1]
    
    if code in x.keys():
        if value in x[code]:
            return 1 
        else:
            return 0 
    else:
        return 0 

i = 0 
for k_v in key_code_value:
    i = i+1
    if i % 100 == 0:
        print(i)
    data_df[k_v] = data_df['检测明细描述_d'].map(lambda x:check_item_code_fun(x,k_v))



100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [60]:
### 保存处理好的车况明细数据
# data_df.to_csv(curr_dir+'/检测报告数据_cyp_fearture_2020-11-12.csv')
data_df = pd.read_csv(curr_dir+'/检测报告数据_cyp_fearture_2020-11-12.csv')

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,89,90,91,92,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
data3m_check = pd.merge(data3m,data_df,on='trade_code',how='left')


In [64]:
## 定义车况明细特征
key_code_value = ['座椅坐垫底部_正常','座椅坐垫底部_水印','座椅坐垫底部_泥沙','座椅扶手_脏污','座椅扶手_破损','左迎宾踏板_破损','左迎宾踏板_改装','左叶子板内衬整形修复_前大框修复','左尾灯框架_重做漆','左尾灯框架_锈蚀','左尾灯框架_切割','左尾灯框架_胶体异常','左尾灯框架_变形','左尾灯框架_钣金修复','左上边梁_重做漆','左前纵梁_严重变形','左前纵梁_锈蚀','左前纵梁_轻微损伤','左前纵梁_切割','左前纵梁_胶体异常','左前纵梁_钣金修复','左前转向灯_破损','左前转向灯_故障','左前转向灯_改装','左前照灯总成_新更换','左前照灯总成_破损','左前照灯总成_老化','左前照灯总成_进水','左前照灯总成_划痕','左前照灯总成_剐蹭','左前照灯总成_故障','左前照灯总成_改装','左前照灯清洗喷嘴罩盖_缺失','左前照灯清洗喷嘴罩盖_破损','左前照灯清洗喷嘴罩盖_改装','左前照灯框架_重做漆','左前照灯框架_锈蚀','左前照灯框架_切割','左前照灯框架_变形','左前照灯框架_钣金修复','左前迎宾灯_缺失','左前迎宾灯_破损','左前迎宾灯_故障','左前迎宾灯_改装','左前翼子板喷漆过_没做好','左前翼子板内衬_重做漆','左前翼子板内衬_严重变形','左前翼子板内衬_锈蚀','左前翼子板内衬_轻微损伤','左前翼子板内衬_切割','左前翼子板内衬_腻子开裂','左前翼子板内衬_螺丝松动','左前翼子板内衬_胶体异常','左前翼子板内衬_胶体开裂','左前翼子板内衬_变形','左前翼子板内衬_钣金修复','左前翼子板灯_缺失','左前翼子板灯_破损','左前翼子板安装不到位_缝隙过大','左前翼子板_重做漆','左前翼子板_虚漆','左前翼子板_锈蚀','左前翼子板_新更换','左前翼子板_色差','左前翼子板_腻子开裂','左前翼子板_螺丝松动','左前翼子板_螺丝拆卸','左前翼子板_流漆','左前翼子板_划痕','左前翼子板_剐蹭','左前翼子板_改装','左前翼子板_掉漆','左前翼子板_底盘装甲','左前翼子板_变形','左前翼子板_爆漆','左前翼子板_钣金修复','左前叶子板内衬_轻微伤','左前悬挂_油污','左前悬挂_新更换','左前悬挂_破损','左前悬挂_改装','左前悬挂_变形','左前雾灯罩_缺失','左前雾灯罩_破损','左前雾灯罩_飞漆','左前雾灯_缺失','左前雾灯_破损','左前雾灯_故障','左前雾灯_改装','左前刹车盘_磨损严重','左前刹车盘_改装','左前刹车卡钳_漏油','左前刹车卡钳_故障','左前刹车卡钳_改装','左前三角玻璃_破损','左前门锁_破损','左前门膜_需要更换','左前门槛饰板_缺失','左前门槛饰板_破损','左前门槛饰板_改装','左前门槛_脏污','左前门槛_锈蚀','左前门槛_流漆','左前门槛_划痕','左前门槛_剐蹭','左前门槛_掉漆','左前门槛_变形','左前门槛_爆漆','左前门更换过_有裂痕','左前门更换过_无原厂喷漆','左前门玻璃升降总开关故障_控制不了其他车门玻璃，','左前轮胎_与登记证不符','左前轮胎_同轴花纹不符','左前轮胎_破损','左前轮胎_磨损','左前轮胎_老化','左前轮眉内衬_缺失','左前轮眉内衬_破损','左前轮眉内侧_飞漆','左前轮眉_色差','左前轮眉_缺失','左前轮眉_破损','左前轮眉_剐蹭','左前轮眉_改装','左前轮毂新更换_左前轮胎新更换','左前轮毂盖_缺失','左前轮毂盖_破损','左前轮毂_剐蹭','左前轮毂_改装','左前轮毂_变形','左前减震器座螺丝松动_维修痕迹','左前减震器座_油污','左前减震器座_严重变形','左前减震器座_锈蚀','左前减震器座_碳化','左前减震器座_轻微损伤','左前减震器座_切割','左前减震器座_胶体异常','左前减震器座_胶体开裂','左前减震器座_变形','左前减震器座_钣金修复','左前减震器_破损','左前减震器_漏油','左前减震器_故障','左前护板破损_无法固定','左前挡泥板_缺失','左前挡泥板_破损','左前大灯框架_轻微损伤','左前大灯更换_大灯进水','左前车门限位器_异响','左前车门限位器_锈蚀','左前车门限位器_破损','左前车门限位器_螺丝松动','左前车门限位器_螺丝拆卸','左前车门限位器_卡滞','左前车门限位器_故障','左前车门限位器_变形','左前车门外拉手_缺失','左前车门外拉手_破损','左前车门外拉手_磨损','左前车门外拉手_划痕','左前车门外拉手_剐蹭','左前车门外拉手_故障','左前车门外拉手_改装','左前车门锁_故障','左前车门锁_改装','左前车门内饰板_脏污','左前车门内饰板_缺失','左前车门内饰板_破损','左前车门内饰板_磨损','左前车门内饰板_老化','左前车门内饰板_划痕','左前车门内饰板_改装','左前车门内侧扣手_缺失','左前车门内侧扣手_破损','左前车门内侧扣手_磨损','左前车门内侧扣手_老化','左前车门内侧扣手_故障','左前车门内侧扣手_改装','左前车门内侧扶手_脏污','左前车门内侧扶手_缺失','左前车门内侧扶手_破损','左前车门内侧扶手_磨损','左前车门内侧扶手_老化','左前车门内侧扶手_划痕','左前车门密封条_缺失','左前车门密封条_破损','左前车门密封条_改装','左前车门铰链_锈蚀','左前车门铰链_切割','左前车门铰链_破损','左前车门铰链_螺丝松动','左前车门铰链_螺丝拆卸','左前车门铰链_变形','左前车门铰链_钣金修复','左前车门封边_重做漆','左前车门封边_胶体异常','左前车门封边_封边不整','左前车门封边_钣金修复','左前车门防擦条_缺失','左前车门防擦条_破损','左前车门防擦条_老化','左前车门防擦条_剐蹭','左前车门防擦条_改装','左前车门_重做漆','左前车门_虚漆','左前车门_锈蚀','左前车门_新更换','左前车门_色差','左前车门_腻子开裂','左前车门_螺丝松动','左前车门_螺丝拆卸','左前车门_流漆','左前车门_划痕','左前车门_剐蹭','左前车门_改装','左前车门_掉漆','左前车门_变形','左前车门_爆漆','左前车窗升降手柄_缺失','左前车窗升降手柄_破损','左前车窗升降手柄_磨损','左前车窗升降开关_缺失','左前车窗升降开关_破损','左前车窗升降开关_磨损','左前车窗玻璃密封条_缺失','左前车窗玻璃密封条_破损','左前车窗玻璃密封条_老化','左前车窗玻璃_异响','左前车窗玻璃_新更换','左前车窗玻璃_破损','左前车窗玻璃_卡滞','左前车窗玻璃_划痕','左前车窗玻璃_故障','左前玻璃压条_锈蚀','左前玻璃压条_缺失','左前玻璃压条_破损','左前玻璃压条_变形','左前半轴球笼防尘套_破损','左前半轴球笼防尘套_漏油','左后座椅头枕_脏污','左后座椅头枕_缺失','左后座椅头枕_破损','左后座椅头枕_改装','左后座椅调节手柄或开关_缺失','左后座椅调节手柄或开关_破损','左后座椅_脏污','左后座椅_烟头烫伤','左后座椅_缺失','左后座椅_破损','左后座椅_磨损','左后座椅_老化','左后座椅_划痕','左后座_儿童桌椅插口内脏污','左后纵梁_严重变形','左后纵梁_锈蚀','左后纵梁_轻微损伤','左后纵梁_切割','左后纵梁_胶体异常','左后纵梁_钣金修复','左后转向灯_破损','左后转向灯_故障','左后迎宾灯_缺失','左后迎宾灯_破损','左后迎宾灯_故障','左后迎宾灯_改装','左后翼子板内饰板_脏污','左后翼子板内饰板_破损','左后翼子板内衬_严重变形','左后翼子板内衬_切割','左后翼子板内衬_变形','左后翼子板内衬_钣金修复','左后翼子板_重做漆','左后翼子板_虚漆','左后翼子板_锈蚀','左后翼子板_新更换','左后翼子板_色差','左后翼子板_切割','左后翼子板_腻子开裂','左后翼子板_流漆','左后翼子板_胶体开裂','左后翼子板_划痕','左后翼子板_剐蹭','左后翼子板_掉漆','左后翼子板_变形','左后翼子板_爆漆','左后翼子板_钣金修复','左后叶子板砂纸印_脏点','左后悬挂_油污','左后悬挂_破损','左后悬挂_螺丝松动','左后悬挂_螺丝拆卸','左后悬挂_改装','左后悬挂_变形','左后尾灯总成_新更换','左后尾灯总成_破损','左后尾灯总成_老化','左后尾灯总成_进水','左后尾灯总成_划痕','左后尾灯总成_剐蹭','左后尾灯总成_故障','左后尾灯总成_改装','左后尾灯油漆_需要抛光','左后刹车盘_改装','左后刹车卡钳_故障','左后刹车卡钳_改装','左后三角玻璃_破损','左后门门板_真皮破损','左后门槛饰板_缺失','左后门槛饰板_破损','左后门槛饰板_改装','左后门槛_锈蚀','左后门槛_磨损','左后门槛_流漆','左后门槛_划痕','左后门槛_剐蹭','左后门槛_掉漆','左后门槛_变形','左后门槛_爆漆','左后轮胎_与登记证不符','左后轮胎_同轴花纹不符','左后轮胎_破损','左后轮胎_磨损','左后轮胎_老化','左后轮眉内衬_缺失','左后轮眉内衬_破损','左后轮眉_色差','左后轮眉_缺失','左后轮眉_破损','左后轮眉_剐蹭','左后轮眉_改装','左后轮眉_挡泥板大量水泥','左后轮毂盖_缺失','左后轮毂盖_破损','左后轮毂_剐蹭','左后轮毂_改装','左后轮毂_变形','左后减震器座_严重变形','左后减震器座_锈蚀','左后减震器座_碳化','左后减震器座_切割','左后减震器座_胶体异常','左后减震器座_胶体开裂','左后减震器座_变形','左后减震器座_钣金修复','左后减震器_破损','左后减震器_漏油','左后减震器_故障','左后导水槽_严重变形','左后导水槽_切割','左后导水槽_腻子开裂','左后导水槽_钣金修复','左后挡泥板_缺失','左后挡泥板_破损','左后车门限位器_异响','左后车门限位器_锈蚀','左后车门限位器_破损','左后车门限位器_螺丝松动','左后车门限位器_螺丝拆卸','左后车门限位器_卡滞','左后车门限位器_故障','左后车门限位器_变形','左后车门外拉手_缺失','左后车门外拉手_破损','左后车门外拉手_磨损','左后车门外拉手_划痕','左后车门外拉手_剐蹭','左后车门外拉手_故障','左后车门外拉手_改装','左后车门锁_故障','左后车门锁_改装','左后车门饰板_破损','左后车门内饰板_脏污','左后车门内饰板_缺失','左后车门内饰板_破损','左后车门内饰板_磨损','左后车门内饰板_老化','左后车门内饰板_划痕','左后车门内饰板_改装','左后车门内侧扣手_缺失','左后车门内侧扣手_破损','左后车门内侧扣手_磨损','左后车门内侧扣手_老化','左后车门内侧扣手_故障','左后车门内侧扣手_改装','左后车门内侧扶手_脏污','左后车门内侧扶手_缺失','左后车门内侧扶手_破损','左后车门内侧扶手_磨损','左后车门内侧扶手_老化','左后车门内侧扶手_划痕','左后车门密封条_缺失','左后车门密封条_破损','左后车门密封条_改装','左后车门铰链_锈蚀','左后车门铰链_切割','左后车门铰链_破损','左后车门铰链_螺丝松动','左后车门铰链_螺丝拆卸','左后车门铰链_变形','左后车门铰链_钣金修复','左后车门封边_重做漆','左后车门封边_胶体异常','左后车门封边_封边不整','左后车门封边_钣金修复','左后车门防擦条_缺失','左后车门防擦条_破损','左后车门防擦条_老化','左后车门防擦条_剐蹭','左后车门防擦条_改装','左后车门_重做漆','左后车门_虚漆','左后车门_锈蚀','左后车门_新更换','左后车门_色差','左后车门_腻子开裂','左后车门_螺丝松动','左后车门_螺丝拆卸','左后车门_流漆','左后车门_划痕','左后车门_剐蹭','左后车门_改装','左后车门_掉漆','左后车门_变形','左后车门_爆漆','左后车窗升降手柄_缺失','左后车窗升降手柄_破损','左后车窗升降手柄_磨损','左后车窗升降开关_缺失','左后车窗升降开关_破损','左后车窗升降开关_磨损','左后车窗改装_远照','左后车窗玻璃密封条_缺失','左后车窗玻璃密封条_破损','左后车窗玻璃密封条_老化','左后车窗玻璃_异响','左后车窗玻璃_新更换','左后车窗玻璃_破损','左后车窗玻璃_卡滞','左后车窗玻璃_划痕','左后车窗玻璃_故障','左后玻璃压条_锈蚀','左后玻璃压条_缺失','左后玻璃压条_破损','左后玻璃压条_变形','左后半轴球笼防尘套_破损','左后半轴球笼防尘套_漏油','左后安全带卡扣_缺失','左后安全带卡扣_破损','左后安全带卡扣_故障','左后安全带_脏污','左后安全带_新更换','左后安全带_水印','左后安全带_破损','左后安全带_故障','左后C_柱内测整形修复','左底大边饰板_锈蚀','左底大边饰板_缺失','左底大边饰板_破损','左底大边饰板_剐蹭','左底大边饰板_改装','左底大边_重做漆','左底大边_锈蚀锈透','左底大边_锈蚀','左底大边_切割','左底大边_腻子开裂','左底大边_划痕','左底大边_剐蹭','左底大边_变形','左底大边_钣金修复','左倒车影像摄像头_破损','左车外后视镜_色差','左车外后视镜_缺失','左车外后视镜_破损','左车外后视镜_卡滞','左车外后视镜_剐蹭','左车外后视镜_故障','左车外后视镜_改装','左车顶密封条_缺失','左车顶密封条_破损','左车顶密封条_变形','左车顶边_重做漆','左车顶边_锈蚀','左车顶边_色差','左车顶边_腻子开裂','左车顶边_划痕','左车顶边_剐蹭','左车顶边_变形','左车顶边_爆漆','左车顶边_钣金修复','左D柱内饰板_破损','左D柱内饰板_划痕','左C柱外侧_虚漆','左C柱外侧_锈蚀','左C柱外侧_色差','左C柱外侧_腻子开裂','左C柱外侧_流漆','左C柱外侧_划痕','左C柱外侧_剐蹭','左C柱外侧_掉漆','左C柱外侧_变形','左C柱外侧_爆漆','左C柱内饰板_脏污','左C柱内饰板_缺失','左C柱内饰板_破损','左C柱内饰板_老化','左C柱内饰板_划痕','左C柱内饰板_改装','左C柱内侧及封边_重做漆','左C柱内侧及封边_严重变形','左C柱内侧及封边_虚漆','左C柱内侧及封边_锈蚀','左C柱内侧及封边_切割','左C柱内侧及封边_流漆','左C柱内侧及封边_划痕','左C柱内侧及封边_剐蹭','左C柱内侧及封边_封边不整','左C柱内侧及封边_掉漆','左C柱内侧及封边_变形','左C柱内侧及封边_爆漆','左C柱内侧及封边_钣金修复','左B柱外侧_虚漆','左B柱外侧_锈蚀','左B柱外侧_色差','左B柱外侧_流漆','左B柱外侧_划痕','左B柱外侧_剐蹭','左B柱外侧_掉漆','左B柱外侧_变形','左B柱外侧_爆漆','左B柱内饰板_脏污','左B柱内饰板_缺失','左B柱内饰板_破损','左B柱内饰板_老化','左B柱内饰板_划痕','左B柱内饰板_改装','左B柱内侧及封边_重做漆','左B柱内侧及封边_严重变形','左B柱内侧及封边_虚漆','左B柱内侧及封边_锈蚀','左B柱内侧及封边_切割','左B柱内侧及封边_流漆','左B柱内侧及封边_划痕','左B柱内侧及封边_剐蹭','左B柱内侧及封边_封边不整','左B柱内侧及封边_掉漆','左B柱内侧及封边_变形','左B柱内侧及封边_爆漆','左B柱内侧及封边_钣金修复','左a柱外侧贴纸_下有改色','左A柱外侧_重做漆','左A柱外侧_虚漆','左A柱外侧_锈蚀','左A柱外侧_色差','左A柱外侧_腻子开裂','左A柱外侧_流漆','左A柱外侧_划痕','左A柱外侧_剐蹭','左A柱外侧_改装','左A柱外侧_掉漆','左A柱外侧_变形','左A柱外侧_爆漆','左A柱内饰板_脏污','左A柱内饰板_缺失','左A柱内饰板_破损','左A柱内饰板_老化','左A柱内饰板_划痕','左A柱内饰板_改装','左A柱内侧及封边_重做漆','左A柱内侧及封边_严重变形','左A柱内侧及封边_虚漆','左A柱内侧及封边_锈蚀','左A柱内侧及封边_切割','左A柱内侧及封边_流漆','左A柱内侧及封边_划痕','左A柱内侧及封边_剐蹭','左A柱内侧及封边_封边不整','左A柱内侧及封边_掉漆','左A柱内侧及封边_变形','左A柱内侧及封边_爆漆','左A柱内侧及封边_钣金修复','左A柱缝隙不均_远照','自动泊车_故障','自动档档把饰板_脏污','自动档档把饰板_破损','自动档档把饰板_磨损','自动档档把饰板_改装','自动档变速器_异响','自动档变速器_螺丝松动','自动档变速器_漏油','自动档变速器_卡滞','自动档变速器_胶体异常','自动档变速器_换挡有冲击','自动档变速器_故障','自动档变速器_闯挡','自动挡挡把饰板_脏污','自动挡挡把饰板_破损','自动挡挡把饰板_磨损','自动挡挡把饰板_改装','自动挡变速器_油液渗漏','自动挡变速器_异响','自动挡变速器_新更换','自动挡变速器_维修痕迹','自动挡变速器_螺丝松动','自动挡变速器_螺丝拆卸','自动挡变速器_漏油','自动挡变速器_卡滞','自动挡变速器_胶体异常','自动挡变速器_换挡有冲击','自动挡变速器_故障','自动挡变速器_改装','自动挡变速器_闯挡','转向柱饰板_脏污','转向柱饰板_缺失','转向柱饰板_破损','转向柱饰板_划痕','转向柱_正常','转向柱_锈蚀','转向柱_水印','转向柱_泥沙','转向助力泵_异响','转向助力泵_漏油','转向助力_异响','转向助力_卡滞','转向助力_故障','转向助力_沉重','转向机防尘套_破损','转向机_漏油','驻车制动_磨损','驻车制动_故障','助力转向油管_漏油','主遮阳板灯盖缺失_','主驾座椅头枕_缺失','主驾座椅头枕_破损','主驾座椅头枕_改装','主驾座椅滑轨_正常','主驾座椅滑轨_锈蚀','主驾座椅滑轨_水印','主驾座椅滑轨_泥沙','主驾座椅滑轨_螺丝异常','主驾座椅功能_故障','主驾座椅调节手柄或开关_缺失','主驾座椅调节手柄或开关_破损','主驾座椅调节方式_故障','主驾座椅_脏污','主驾座椅_烟头烫伤','主驾座椅_缺失','主驾座椅_破损','主驾座椅_磨损','主驾座椅_老化','主驾座椅_划痕','主驾座椅_故障','主驾座椅_改装','主驾坐垫下方铁皮_正常','主驾遮阳板_脏污','主驾遮阳板_缺失','主驾遮阳板_破损','主驾阅读灯_缺失','主驾阅读灯_破损','主驾阅读灯_故障','主驾驶位_加装氛围灯','主驾驶门锁眼_破损','主驾驶_仪表下方锈蚀','主驾化妆镜灯_故障','主驾化妆镜_缺失','主驾化妆镜_破损','主驾调节方式_故障','主驾安全带卡扣_缺失','主驾安全带卡扣_破损','主驾安全带卡扣_故障','主驾安全带_正常','主驾安全带_脏污','主驾安全带_新更换','主驾安全带_水印','主驾安全带_破损','主驾安全带_故障','中央集控_破损','中央集控_磨损','中央集控_故障','中网新更换_老化','中冷器挤压_需拆解确认','中控锁_故障','正时盖板螺丝松动_机脚螺丝松动','整体右侧漆面两遍漆，漆面不平鱼鳞纹_需重做','阅读灯_故障','元宝梁_严重变形','元宝梁_新更换','元宝梁_托底','元宝梁_变形','雨刷喷水壶_破损','雨刷喷水壶_漏水','雨刮组合开关_破损','雨刮组合开关_故障','雨刮水管_脱落','右迎宾踏板_破损','右迎宾踏板_改装','右尾灯框架_重做漆','右尾灯框架_锈蚀','右尾灯框架_切割','右尾灯框架_胶体异常','右尾灯框架_变形','右尾灯框架_钣金修复','右前纵梁缓冲块更换_螺丝松动痕迹','右前纵梁_严重变形','右前纵梁_锈蚀','右前纵梁_轻微损伤','右前纵梁_切割','右前纵梁_胶体异常','右前纵梁_钣金修复','右前转向灯_破损','右前转向灯_故障','右前转向灯_改装','右前照灯总成_新更换','右前照灯总成_破损','右前照灯总成_老化','右前照灯总成_进水','右前照灯总成_划痕','右前照灯总成_剐蹭','右前照灯总成_故障','右前照灯总成_改装','右前照灯清洗喷嘴罩盖_缺失','右前照灯清洗喷嘴罩盖_破损','右前照灯清洗喷嘴罩盖_改装','右前照灯框架_重做漆','右前照灯框架_锈蚀','右前照灯框架_切割','右前照灯框架_变形','右前照灯框架_钣金修复','右前迎宾灯_缺失','右前迎宾灯_破损','右前迎宾灯_故障','右前迎宾灯_改装','右前翼子板内衬_重做漆','右前翼子板内衬_严重变形','右前翼子板内衬_锈蚀','右前翼子板内衬_轻微损伤','右前翼子板内衬_切割','右前翼子板内衬_腻子开裂','右前翼子板内衬_胶体异常','右前翼子板内衬_胶体开裂','右前翼子板内衬_变形','右前翼子板内衬_钣金修复','右前翼子板灯_缺失','右前翼子板灯_破损','右前翼子板_重做漆','右前翼子板_脏污','右前翼子板_虚漆','右前翼子板_锈蚀','右前翼子板_新更换','右前翼子板_色差','右前翼子板_前杠翼子灰','右前翼子板_腻子开裂','右前翼子板_螺丝松动','右前翼子板_螺丝拆卸','右前翼子板_流漆','右前翼子板_划痕','右前翼子板_剐蹭','右前翼子板_改装','右前翼子板_掉漆','右前翼子板_底盘装甲','右前翼子板_变形','右前翼子板_爆漆','右前翼子板_钣金修复','右前叶子板_泥沙','右前悬挂_油污','右前悬挂_新更换','右前悬挂_破损','右前悬挂_改装','右前悬挂_变形','右前雾灯罩_雾漆','右前雾灯罩_缺失','右前雾灯罩_破损','右前雾灯罩_飞漆','右前雾灯_破损','右前雾灯_故障','右前雾灯_改装','右前刹车盘_磨损严重','右前刹车盘_改装','右前刹车卡钳_故障','右前刹车卡钳_改装','右前三角玻璃_破损','右前门饰板_划痕','右前门槛饰板_缺失','右前门槛饰板_破损','右前门槛饰板_改装','右前门槛_锈蚀','右前门槛_磨损','右前门槛_流漆','右前门槛_划痕','右前门槛_剐蹭','右前门槛_掉漆','右前门槛_变形','右前门槛_爆漆','右前轮胎_与登记证不符','右前轮胎_同轴花纹不符','右前轮胎_破损','右前轮胎_磨损','右前轮胎_老化','右前轮内衬_新更换','右前轮眉内衬_缺失','右前轮眉内衬_破损','右前轮眉_色差','右前轮眉_缺失','右前轮眉_破损','右前轮眉_剐蹭','右前轮眉_改装','右前轮毂盖_缺失','右前轮毂盖_破损','右前轮毂_剐蹭','右前轮毂_改装','右前轮毂_变形','右前减震器座_严重变形','右前减震器座_锈蚀','右前减震器座_碳化','右前减震器座_轻微损伤','右前减震器座_切割','右前减震器座_胶体异常','右前减震器座_胶体开裂','右前减震器座_变形','右前减震器座_钣金修复','右前减震器_破损','右前减震器_漏油','右前减震器_故障','右前挡泥板_缺失','右前挡泥板_破损','右前大灯_松动','右前车门限位器_异响','右前车门限位器_锈蚀','右前车门限位器_破损','右前车门限位器_螺丝松动','右前车门限位器_螺丝拆卸','右前车门限位器_卡滞','右前车门限位器_故障','右前车门限位器_变形','右前车门外拉手_缺失','右前车门外拉手_破损','右前车门外拉手_磨损','右前车门外拉手_划痕','右前车门外拉手_剐蹭','右前车门外拉手_故障','右前车门外拉手_改装','右前车门锁_故障','右前车门锁_改装','右前车门内饰板_脏污','右前车门内饰板_缺失','右前车门内饰板_破损','右前车门内饰板_磨损','右前车门内饰板_老化','右前车门内饰板_划痕','右前车门内饰板_改装','右前车门内侧扣手_缺失','右前车门内侧扣手_破损','右前车门内侧扣手_磨损','右前车门内侧扣手_老化','右前车门内侧扣手_故障','右前车门内侧扣手_改装','右前车门内侧扶手_脏污','右前车门内侧扶手_缺失','右前车门内侧扶手_破损','右前车门内侧扶手_磨损','右前车门内侧扶手_老化','右前车门内侧扶手_划痕','右前车门密封条_缺失','右前车门密封条_破损','右前车门密封条_改装','右前车门铰链_锈蚀','右前车门铰链_切割','右前车门铰链_破损','右前车门铰链_螺丝松动','右前车门铰链_螺丝拆卸','右前车门铰链_变形','右前车门铰链_钣金修复','右前车门封边_重做漆','右前车门封边_胶体异常','右前车门封边_封边不整','右前车门封边_钣金修复','右前车门防擦条_缺失','右前车门防擦条_破损','右前车门防擦条_老化','右前车门防擦条_剐蹭','右前车门防擦条_改装','右前车门_重做漆','右前车门_虚漆','右前车门_锈蚀','右前车门_新更换','右前车门_色差','右前车门_腻子开裂','右前车门_螺丝松动','右前车门_螺丝拆卸','右前车门_流漆','右前车门_划痕','右前车门_剐蹭','右前车门_改装','右前车门_掉漆','右前车门_变形','右前车门_爆漆','右前车窗升降手柄_缺失','右前车窗升降手柄_破损','右前车窗升降手柄_磨损','右前车窗升降开关_缺失','右前车窗升降开关_破损','右前车窗升降开关_磨损','右前车窗玻璃密封条_缺失','右前车窗玻璃密封条_破损','右前车窗玻璃密封条_老化','右前车窗玻璃_异响','右前车窗玻璃_新更换','右前车窗玻璃_破损','右前车窗玻璃_卡滞','右前车窗玻璃_划痕','右前车窗玻璃_故障','右前玻璃压条_锈蚀','右前玻璃压条_缺失','右前玻璃压条_破损','右前玻璃压条_变形','右前半轴球笼防尘套_破损','右前半轴球笼防尘套_漏油','右前半轴断裂_右前下摆臂变形','右前a柱贴纸_划痕','右后座椅头枕_脏污','右后座椅头枕_缺失','右后座椅头枕_破损','右后座椅头枕_改装','右后座椅调节手柄或开关_缺失','右后座椅调节手柄或开关_破损','右后座椅_脏污','右后座椅_烟头烫伤','右后座椅_缺失','右后座椅_破损','右后座椅_磨损','右后座椅_老化','右后座椅_划痕','右后纵梁_严重变形','右后纵梁_锈蚀','右后纵梁_轻微损伤','右后纵梁_切割','右后纵梁_胶体异常','右后纵梁_变形','右后纵梁_钣金修复','右后转向灯_破损','右后转向灯_故障','右后转向灯_改装','右后迎宾灯_缺失','右后迎宾灯_破损','右后迎宾灯_故障','右后迎宾灯_改装','右后翼子板内饰板_脏污','右后翼子板内饰板_破损','右后翼子板内衬_严重变形','右后翼子板内衬_切割','右后翼子板内衬_变形','右后翼子板内衬_钣金修复','右后翼子板剐蹭变形_需做钣金','右后翼子板_重做漆','右后翼子板_右后门划伤','右后翼子板_虚漆','右后翼子板_锈蚀','右后翼子板_新更换','右后翼子板_贴纸','右后翼子板_色差','右后翼子板_切割','右后翼子板_腻子开裂','右后翼子板_流漆','右后翼子板_胶体开裂','右后翼子板_划痕','右后翼子板_剐蹭','右后翼子板_掉漆','右后翼子板_变形','右后翼子板_爆漆','右后翼子板_钣金修复','右后叶子_右后门已处理','右后悬挂_油污','右后悬挂_新更换','右后悬挂_破损','右后悬挂_改装','右后悬挂_变形','右后尾灯总成_新更换','右后尾灯总成_破损','右后尾灯总成_老化','右后尾灯总成_进水','右后尾灯总成_划痕','右后尾灯总成_剐蹭','右后尾灯总成_故障','右后尾灯总成_改装','右后视镜虚漆_锈点','右后刹车盘_改装','右后刹车卡钳_漏油','右后刹车卡钳_故障','右后刹车卡钳_改装','右后三角玻璃镀铬条_划痕','右后三角玻璃_破损','右后门饰板_破损','右后门饰板_划痕','右后门槛饰板_缺失','右后门槛饰板_破损','右后门槛饰板_改装','右后门槛_锈蚀','右后门槛_磨损','右后门槛_流漆','右后门槛_划痕','右后门槛_剐蹭','右后门槛_掉漆','右后门槛_变形','右后门槛_爆漆','右后门顶棚扶手破损_缺失','右后轮胎_与登记证不符','右后轮胎_同轴花纹不符','右后轮胎_破损','右后轮胎_磨损','右后轮胎_老化','右后轮眉内衬_缺失','右后轮眉内衬_破损','右后轮眉_色差','右后轮眉_缺失','右后轮眉_破损','右后轮眉_剐蹭','右后轮眉_更换','右后轮眉_改装','右后轮毂盖_缺失','右后轮毂盖_破损','右后轮毂_剐蹭','右后轮毂_改装','右后轮毂_变形','右后流水槽钣金_锈穿','右后减震器座_严重变形','右后减震器座_锈蚀','右后减震器座_碳化','右后减震器座_切割','右后减震器座_胶体异常','右后减震器座_胶体开裂','右后减震器座_变形','右后减震器座_钣金修复','右后减震器_破损','右后减震器_漏油','右后减震器_故障','右后后减震，右后悬挂上连杆总成_和，右后羊角更换','右后导水槽_严重变形','右后导水槽_切割','右后导水槽_腻子开裂','右后导水槽_钣金修复','右后挡泥板_缺失','右后挡泥板_破损','右后车门限位器_异响','右后车门限位器_锈蚀','右后车门限位器_破损','右后车门限位器_螺丝松动','右后车门限位器_螺丝拆卸','右后车门限位器_卡滞','右后车门限位器_故障','右后车门限位器_变形','右后车门外拉手_缺失','右后车门外拉手_破损','右后车门外拉手_磨损','右后车门外拉手_划痕','右后车门外拉手_剐蹭','右后车门外拉手_故障','右后车门外拉手_改装','右后车门锁_故障','右后车门锁_改装','右后车门内饰板_脏污','右后车门内饰板_缺失','右后车门内饰板_破损','右后车门内饰板_磨损','右后车门内饰板_老化','右后车门内饰板_划痕','右后车门内饰板_改装','右后车门内侧扣手_缺失','右后车门内侧扣手_破损','右后车门内侧扣手_磨损','右后车门内侧扣手_老化','右后车门内侧扣手_故障','右后车门内侧扣手_改装','右后车门内侧扶手_脏污','右后车门内侧扶手_缺失','右后车门内侧扶手_破损','右后车门内侧扶手_磨损','右后车门内侧扶手_老化','右后车门内侧扶手_划痕','右后车门密封条_缺失','右后车门密封条_破损','右后车门密封条_改装','右后车门铰链_锈蚀','右后车门铰链_切割','右后车门铰链_破损','右后车门铰链_螺丝松动','右后车门铰链_螺丝拆卸','右后车门铰链_变形','右后车门铰链_钣金修复','右后车门封边_重做漆','右后车门封边_胶体异常','右后车门封边_封边不整','右后车门封边_钣金修复','右后车门防擦条_缺失','右后车门防擦条_破损','右后车门防擦条_老化','右后车门防擦条_剐蹭','右后车门防擦条_改装','右后车门玻璃亮条_氧化','右后车门_重做漆','右后车门_虚漆','右后车门_锈蚀','右后车门_新更换','右后车门_色差','右后车门_腻子开裂','右后车门_螺丝松动','右后车门_螺丝拆卸','右后车门_流漆','右后车门_划痕','右后车门_剐蹭','右后车门_改装','右后车门_掉漆','右后车门_变形','右后车门_爆漆','右后车窗升降手柄_缺失','右后车窗升降手柄_破损','右后车窗升降手柄_磨损','右后车窗升降开关_缺失','右后车窗升降开关_破损','右后车窗升降开关_磨损','右后车窗玻璃密封条_缺失','右后车窗玻璃密封条_破损','右后车窗玻璃密封条_老化','右后车窗玻璃改装_远照','右后车窗玻璃_异响','右后车窗玻璃_新更换','右后车窗玻璃_破损','右后车窗玻璃_卡滞','右后车窗玻璃_划痕','右后车窗玻璃_故障','右后玻璃压条_锈蚀','右后玻璃压条_缺失','右后玻璃压条_破损','右后玻璃压条_变形','右后半轴球笼防尘套_破损','右后半轴球笼防尘套_漏油','右后安全带卡扣_缺失','右后安全带卡扣_破损','右后安全带卡扣_故障','右后安全带_脏污','右后安全带_新更换','右后安全带_水印','右后安全带_破损','右后安全带_故障','右底大边饰板_锈蚀','右底大边饰板_缺失','右底大边饰板_破损','右底大边饰板_剐蹭','右底大边饰板_改装','右底大边_重做漆','右底大边_锈蚀锈透','右底大边_锈蚀','右底大边_切割','右底大边_腻子开裂','右底大边_划痕','右底大边_剐蹭','右底大边_变形','右底大边_钣金修复','右倒车影像摄像头_破损','右车外后视镜_色差','右车外后视镜_缺失','右车外后视镜_破损','右车外后视镜_卡滞','右车外后视镜_剐蹭','右车外后视镜_故障','右车外后视镜_改装','右车顶密封条_缺失','右车顶密封条_破损','右车顶密封条_变形','右车顶边_重做漆','右车顶边_锈蚀','右车顶边_色差','右车顶边_腻子开裂','右车顶边_划痕','右车顶边_剐蹭','右车顶边_变形','右车顶边_爆漆','右车顶边_钣金修复','右侧裙板_新更换','右D柱内饰板_脏污','右D柱内饰板_破损','右D柱内饰板_划痕','右C柱外侧_虚漆','右C柱外侧_锈蚀','右C柱外侧_色差','右C柱外侧_腻子开裂','右C柱外侧_流漆','右C柱外侧_划痕','右C柱外侧_剐蹭','右C柱外侧_掉漆','右C柱外侧_变形','右C柱外侧_爆漆','右C柱内饰板_脏污','右C柱内饰板_缺失','右C柱内饰板_破损','右C柱内饰板_老化','右C柱内饰板_划痕','右C柱内饰板_改装','右C柱内侧及封边_重做漆','右C柱内侧及封边_严重变形','右C柱内侧及封边_虚漆','右C柱内侧及封边_锈蚀','右C柱内侧及封边_切割','右C柱内侧及封边_流漆','右C柱内侧及封边_划痕','右C柱内侧及封边_剐蹭','右C柱内侧及封边_封边不整','右C柱内侧及封边_掉漆','右C柱内侧及封边_变形','右C柱内侧及封边_爆漆','右C柱内侧及封边_钣金修复','右B柱外侧_虚漆','右B柱外侧_锈蚀','右B柱外侧_色差','右B柱外侧_流漆','右B柱外侧_划痕','右B柱外侧_剐蹭','右B柱外侧_掉漆','右B柱外侧_变形','右B柱外侧_爆漆','右B柱内饰板_脏污','右B柱内饰板_缺失','右B柱内饰板_破损','右B柱内饰板_老化','右B柱内饰板_划痕','右B柱内饰板_改装','右B柱内侧及封边_重做漆','右B柱内侧及封边_严重变形','右B柱内侧及封边_虚漆','右B柱内侧及封边_锈蚀','右B柱内侧及封边_切割','右B柱内侧及封边_流漆','右B柱内侧及封边_划痕','右B柱内侧及封边_剐蹭','右B柱内侧及封边_封边不整','右B柱内侧及封边_掉漆','右B柱内侧及封边_变形','右B柱内侧及封边_爆漆','右B柱内侧及封边_钣金修复','右A柱外侧_重做漆','右A柱外侧_虚漆','右A柱外侧_锈蚀','右A柱外侧_色差','右A柱外侧_腻子开裂','右A柱外侧_流漆','右A柱外侧_划痕','右A柱外侧_剐蹭','右A柱外侧_改装','右A柱外侧_掉漆','右A柱外侧_变形','右A柱外侧_爆漆','右A柱内饰板_脏污','右A柱内饰板_缺失','右A柱内饰板_破损','右A柱内饰板_老化','右A柱内饰板_划痕','右A柱内饰板_改装','右A柱内侧及封边_重做漆','右A柱内侧及封边_严重变形','右A柱内侧及封边_虚漆','右A柱内侧及封边_锈蚀','右A柱内侧及封边_切割','右A柱内侧及封边_流漆','右A柱内侧及封边_划痕','右A柱内侧及封边_剐蹭','右A柱内侧及封边_封边不整','右A柱内侧及封边_掉漆','右A柱内侧及封边_变形','右A柱内侧及封边_爆漆','右A柱内侧及封边_钣金修复','油箱盖开关_破损','油箱盖开关_故障','油箱盖_锈蚀','油箱盖_贴纸','油箱盖_色差','油箱盖_缺失','油箱盖_划痕','油箱盖_剐蹭','油箱盖_变形','油漆已做完_清洗完毕','油底壳_半轴更换','影音设备_缺失','影音设备_破损','影音设备_磨损','影音设备_故障','音响线_加装','音响控制面板_破损','仪表盘外壳_脏污','仪表盘外壳_破损','仪表盘外壳_划痕','仪表盘外壳_改装','仪表盘背景灯_故障','仪表_破损','仪表_故障灯亮','仪表_故障','一键启动_故障','钥匙_缺失','钥匙_破损','钥匙_磨损','钥匙_故障','钥匙_改装','扬声器_水印','眼镜盒_缺失','眼镜盒_故障','蓄电池_锈蚀','蓄电池_破损','蓄电池_漏水','蓄电池_故障','锈蚀_变形','行李箱外拉手_缺失','行李箱外拉手_破损','行李箱外拉手_磨损','行李箱外拉手_划痕','行李箱外拉手_故障','行李箱锁_故障','行李箱内饰板_脏污','行李箱内饰板_缺失','行李箱内饰板_破损','行李箱内饰板_划痕','行李箱内饰板_改装','行李箱内拉手_缺失','行李箱内拉手_破损','行李箱内拉手_磨损','行李箱内拉手_划痕','行李箱内拉手_故障','行李箱内拉手_改装','行李箱密封条_缺失','行李箱密封条_破损','行李箱隔板_锈蚀','行李箱隔板_切割','行李箱隔板_划痕','行李箱隔板_变形','行李箱隔板_钣金修复','行李箱盖支撑杆_锈蚀','行李箱盖支撑杆_缺失','行李箱盖支撑杆_破损','行李箱盖支撑杆_磨损','行李箱盖支撑杆_漏油','行李箱盖支撑杆_故障','行李箱盖支撑杆_改装','行李箱盖内饰板_脏污','行李箱盖内饰板_缺失','行李箱盖内饰板_破损','行李箱盖内饰板_划痕','行李箱盖内饰板_剐蹭','行李箱盖内饰板_改装','行李箱盖内衬_重做漆','行李箱盖内衬_锈蚀','行李箱盖内衬_胶体异常','行李箱盖内衬_划痕','行李箱盖内衬_封边不整','行李箱盖内衬_变形','行李箱盖内衬_钣金修复','行李箱盖内边缘_重做漆','行李箱盖内边缘_锈蚀','行李箱盖内边缘_胶体异常','行李箱盖内边缘_划痕','行李箱盖内边缘_封边不整','行李箱盖内边缘_变形','行李箱盖内边缘_钣金修复','行李箱盖铰链_锈蚀','行李箱盖铰链_切割','行李箱盖铰链_螺丝松动','行李箱盖铰链_螺丝拆卸','行李箱盖铰链_变形','行李箱盖铰链_钣金修复','行李箱盖_重做漆','行李箱盖_脏污','行李箱盖_虚漆','行李箱盖_锈蚀','行李箱盖_新更换','行李箱盖_色差','行李箱盖_腻子开裂','行李箱盖_螺丝松动','行李箱盖_螺丝拆卸','行李箱盖_流漆','行李箱盖_划痕','行李箱盖_剐蹭','行李箱盖_改装','行李箱盖_掉漆','行李箱盖_变形','行李箱盖_爆漆','行李箱盖_雹子坑','行李箱封边_严重变形','行李箱封边_锈蚀','行李箱封边_切割','行李箱封边_胶体异常','行李箱封边_封边不整','行李箱封边_变形','行李箱封边_钣金修复','行李箱底板四边_水印','行李箱底板四边_泥沙','行李箱底板四边_胶体异常','行李箱底板四边_胶体开裂','行李箱底板_重做漆','行李箱底板_脏污','行李箱底板_严重变形','行李箱底板_锈蚀','行李箱底板_碳化','行李箱底板_水印','行李箱底板_切割','行李箱底板_泥沙','行李箱底板_胶体开裂','行李箱底板_变形','行李箱底板_钣金修复','行李箱灯_缺失','行李箱灯_破损','行李箱储物隔板_脏污','行李箱储物隔板_缺失','行李箱储物隔板_破损','行李箱储物隔板_改装','行李箱储物隔板_变形','行李架左侧氧化成_色差','行李架右侧氧化成_色差','行李架_缺失','行李架_破损','行李架_改装','行车灯_改装','新更换_钣金','新包坐垫_座椅是否破损未知','雾灯开关_破损','雾灯开关_磨损','无法正常检测芯片_可能电池问题','尾箱盖_油箱盖拉手塑料套破损','铁牌架_加装','贴牌架_加装','天线_缺失','天线_破损','天线_改装','天窗遮阳帘_脏污','天窗遮阳帘_缺失','天窗遮阳帘_破损','天窗遮阳帘_卡滞','天窗遮阳帘_故障','天窗遮阳板脏污_水印','天窗开关_破损','天窗开关_磨损','天窗打胶_无法开启','天窗_异响','天窗_破损','天窗_卡滞','天窗_划痕','天窗_故障','锁块衬板重做漆_未有严重损伤','随车慢充充电器_损伤','随车慢充充电器_故障','随车工具_锈蚀','随车工具_缺失','随车工具_底座','四只轮毂_轮胎改装','四条轮胎磨损到极限_需更换','四门门框有胶_体贴纸','四门把手加装饰条_磨损掉漆','水箱左下支架连接处开裂_但水箱整体框架未更换','水箱下支架_重做漆','水箱下支架_严重变形','水箱下支架_锈蚀','水箱下支架_轻微损伤','水箱下支架_切割','水箱下支架_空调管道有飞漆','水箱下支架_钣金修复','水箱上支架_重做漆','水箱上支架_严重变形','水箱上支架_锈蚀','水箱上支架_新更换','水箱上支架_轻微损伤','水箱上支架_切割','水箱上支架_螺丝松动','水箱上支架_螺丝拆卸','水箱上支架_钣金修复','水箱上护板_缺失','水箱上护板_破损','水箱上护板_螺丝拧动','水箱上护板_改装','水箱里有油_','水箱副水壶_破损','水箱副水壶_漏水','水箱_锈蚀','水箱_新更换','水箱_破损','水箱_漏水','水箱_冷凝器','水箱_电子扇冷凝器更换','水箱_电子扇','水箱_变形','手套箱灯_故障','手套箱_脏污','手套箱_破损','手套箱_磨损','手套箱_划痕','手套箱_故障','手套箱_改装','手刹_破损脏污','手动档变速器_异响','手动档变速器_新更换','手动档变速器_螺丝松动','手动档变速器_漏油','手动档变速器_卡滞','手动档变速器_胶体异常','手动档变速器_故障','手动挡变速器_油液渗漏','手动挡变速器_异响','手动挡变速器_新更换','手动挡变速器_维修痕迹','手动挡变速器_螺丝松动','手动挡变速器_漏油','手动挡变速器_卡滞','手动挡变速器_胶体异常','手动挡变速器_故障','手动挡变速器_改装','手动变速器漏油异响抖动_挂挡冲击','事故理赔详情_','刹车总泵_漏油','刹车总泵_故障','刹车助力泵_锈蚀','刹车液_液位异常','刹车液_液体变质','刹车踏板_锈蚀','刹车踏板_破损','刹车踏板_磨损','刹车踏板_踩塌延迟','刹车磨损严重_不灵敏','散热风扇_破损','散热风扇_故障','扰流板（后尾翼）_色差','扰流板（后尾翼）_缺失','扰流板（后尾翼）_破损','扰流板（后尾翼）_划痕','扰流板（后尾翼）_改装','驱动类型_故障','前中网掉漆_虚漆','前照灯雨刷器_锈蚀','前照灯雨刷器_破损','前照灯雨刷器_故障','前照灯雨刷器_改装','前照灯清洗喷嘴_故障','前雾灯_进水','前围上盖板_缺失','前围上盖板_破损','前围挡板（防火墙）_重做漆','前围挡板（防火墙）_严重变形','前围挡板（防火墙）_锈蚀','前围挡板（防火墙）_轻微损伤','前围挡板（防火墙）_切割','前围挡板（防火墙）_胶体异常','前围挡板（防火墙）_胶体开裂','前围挡板（防火墙）_划痕','前围挡板（防火墙）_变形','前围挡板（防火墙）_钣金修复','前拖钩罩盖_缺失','前拖钩罩盖_破损','前拖钩_变形','前排烟灰缸底座_泥沙','前排烟灰缸_缺失','前排烟灰缸_破损','前排空调出风口_脏污','前排空调出风口_缺失','前排空调出风口_破损','前排空调出风口_故障','前排空调出风口_改装','前后胎品牌_花纹不一致','前后拍照以对换_变形','前杠下托架挡泥皮_破损','前杠下格栅_缺失','前杠下格栅_破损','前杠下格栅_飞漆','前杠_摸腻子','前风挡雨刷器_锈蚀','前风挡雨刷器_破损','前风挡雨刷器_故障','前风挡雨刷器_改装','前风挡下沿_锈蚀','前风挡下沿_腻子开裂','前风挡下沿_磨损','前风挡下沿_流漆','前风挡下沿_剐蹭','前风挡下沿_掉漆','前风挡下沿_变形','前风挡下沿_爆漆','前风挡清洗喷嘴_故障','前风挡密封条_缺失','前风挡密封条_破损','前防撞梁_严重变形','前防撞梁_锈蚀','前防撞梁_新更换','前防撞梁_螺丝松动','前防撞梁_变形','前防撞梁_钣金修复','前倒车影像摄像头_缺失','前倒车影像摄像头_破损','前倒车雷达探头_缺失','前倒车雷达探头_破损','前挡风玻璃_新更换','前挡风玻璃_碎点','前挡风玻璃_破损','前挡风玻璃_划痕','前挡风玻璃_改装','前车标_锈蚀','前车标_缺失','前车标_破损','前车标_老化','前车标_改装','前保险杠装饰板_破损','前保险杠装饰板_剐蹭','前保险杠支架_锈蚀','前保险杠支架_破损','前保险杠支架_剐蹭','前保险杠支架_变形','前保险杠有沥青_树脂','前保险杠螺丝有过拆卸_螺丝更换','前保险杠_重做漆','前保险杠_新更换','前保险杠_色差','前保险杠_破损','前保险杠_腻子开裂','前保险杠_划痕','前保险杠_剐蹭','前保险杠_改装','前保险杠_缝隙不匀','前保险杠_飞漆','前保险杠_掉漆','前保险杠_变形','前保下饰条_左中右已更换','千斤顶_扳手','气囊缺失_喇叭故障','气囊故灯_故障','气囊_新更换','气囊_螺丝松动','气囊_缝隙不匀','启停_故障','启动检测芯片信号差_遥控器无法锁门','启动机（马达）_异响','启动机（马达）_故障','皮带_缺失','皮带_破损','皮带_老化','泡水检测正常。工作台支架_保险盒','泡水检测正常。点火锁支架_方向机操作杠','排气歧管_破损','排气管支架_锈蚀','排气管支架_破损','排气管支架_改装','排气管支架_变形','排气管消音器_锈蚀','排气管消音器_破损','排气管消音器_改装','排气管消音器_变形','排气管尾喉_锈蚀','排气管尾喉_改装','排气管_锈蚀','排气管_破损','排气管_改装','排气管_变形','拍照刮擦_变形','逆变器_螺丝松动','内饰_脏污','内饰_线路改装','内饰_破损','内蓬顶变形_脏污','铭牌区少一块_被撕了','门饰条剐蹭_掉漆','轮毂_翼子板已修复','龙门架_重做漆','离合器_异响','离合器_故障','冷凝器_破损','冷凝器_漏油','喇叭_故障','喇叭_改装','库存时间过长_左前杠有掉漆','控制器_螺丝松动','空气悬架_故障','空气滤芯盒固定不牢_异响','空滤总成新更换_空滤缺失','空调制热效果不好_缓慢','空调压缩机_脏污','空调压缩机_异响','空调压缩机_破损','空调压缩机_漏油','空调压缩机_故障','空调鼓风机_异响','空调鼓风机_故障','空调高压管_盖子缺失','空调按钮_缺失','空调按钮_破损','空调按钮_磨损','空调按钮_卡滞','空调按钮_故障','空调_故障','可拆水箱框架_锈蚀','可拆水箱框架_新更换','可拆水箱框架_破损','可拆水箱框架_螺丝松动','进气歧管_破损','进气盒卡扣断裂_无法固定','进气格栅（中网）_破损','进气格栅（中网）_改装','减档换挡拨片_破损','减档换挡拨片_磨损','减档换挡拨片_故障','减档换挡拨片_改装','减挡换挡拨片_磨损','减挡换挡拨片_故障','减挡换挡拨片_改装','加装座椅记忆_故障','加装_多媒体音量控制按键','加档换挡拨片_破损','加档换挡拨片_磨损','加档换挡拨片_故障','加档换挡拨片_改装','加挡换挡拨片_破损','加挡换挡拨片_故障','加挡换挡拨片_改装','机油需要更换_胎压监测报警','机油乳化_防冻液进入燃烧室','机油故灯_故障','机油尺_破损','机油_液位异常','机油_液体变质','机铰螺丝松动_机铰垫新跟换','机盖开启挤碰左前翼子板_','机盖飞漆_氧化','机盖调整垫_缺失','机舱内锈蚀_泥沙','机舱内_饰板破损','后行李箱盖饰条_缺失','后行李箱盖饰条_破损','后行李箱盖饰条_老化','后雾灯_破损','后雾灯_改装','后围板_重做漆','后围板_严重变形','后围板_锈蚀','后围板_切割','后围板_胶体异常','后围板_划痕','后围板_变形','后围板_钣金修复','后拖钩罩盖_缺失','后拖钩罩盖_破损','后拖钩_变形','后视镜调节_破损','后视镜_上下调节故障','后视镜_调节开关磨损','后排座椅中央扶手_脏污','后排座椅中央扶手_破损','后排座椅中央扶手_磨损','后排座椅中央扶手_划痕','后排座椅中央扶手_改装','后排座椅破损故障_支架断裂','后排座椅磨损_老化','后排座椅开裂_破损','后排座椅功能_故障','后排座椅变形_影响过户买方负责','后排座椅按键故障_加热故障','后排音响面板_破损','后排烟灰缸底座_泥沙','后排烟灰缸_缺失','后排烟灰缸_破损','后排空调出风口_脏污','后排空调出风口_缺失','后排空调出风口_破损','后排空调出风口_故障','后排空调出风口_改装','后排出风口破损_饰板缺失','后排USB_接囗破损','后加装一键启动_不好使','后杠_后盖','后盖_后杠','后风挡雨刷器_锈蚀','后风挡雨刷器_破损','后风挡雨刷器_故障','后风挡雨刷器_改装','后风挡下沿_锈蚀','后风挡下沿_腻子开裂','后风挡下沿_流漆','后风挡下沿_剐蹭','后风挡下沿_掉漆','后风挡下沿_变形','后风挡下沿_爆漆','后风挡清洗喷嘴_故障','后风挡密封条_缺失','后风挡密封条_破损','后防撞梁钣金_更换','后防撞梁_锈蚀','后防撞梁_新更换','后防撞梁_螺丝松动','后防撞梁_变形','后防撞梁_钣金修复','后倒车影像摄像头_缺失','后倒车影像摄像头_破损','后倒车雷达探头_缺失','后倒车雷达探头_破损','后挡风玻璃贴膜破损_。','后挡风玻璃_新更换','后挡风玻璃_碎点','后挡风玻璃_破损','后挡风玻璃_划痕','后挡风玻璃_改装','后车灯贴纸_','后车窗遮阳帘_异响','后车窗遮阳帘_卡滞','后车窗遮阳帘_故障','后车标_锈蚀','后车标_缺失','后车标_破损','后车标_老化','后车标_改装','后备箱内加帖隔音_','后备箱开启方式_故障','后备箱_右后翼不合缝','后备箱_右后翼','后保险杠装饰板_破损','后保险杠装饰板_划痕','后保险杠装饰板_剐蹭','后保险杠支架_锈蚀','后保险杠支架_剐蹭','后保险杠支架_变形','后保险杠反光板_缺失','后保险杠反光板_破损','后保险杠_重做漆','后保险杠_新更换','后保险杠_色差','后保险杠_破损','后保险杠_腻子开裂','后保险杠_划痕','后保险杠_剐蹭','后保险杠_改装','后保险杠_缝隙不匀','后保险杠_掉漆','后保险杠_变形','后保_后保下杠已处理','工作台铁部件_锈蚀','工作台_脏污','工作台_新更换','工作台_破损','工作台_螺丝松动','工作台_划痕','工作台_改装','工作台_缝隙不匀','副驾座椅头枕_缺失','副驾座椅头枕_破损','副驾座椅头枕_改装','副驾座椅滑轨_正常','副驾座椅滑轨_锈蚀','副驾座椅滑轨_水印','副驾座椅滑轨_泥沙','副驾座椅功能_故障','副驾座椅调节手柄或开关_缺失','副驾座椅调节手柄或开关_破损','副驾座椅调节方式_故障','副驾座椅_脏污','副驾座椅_烟头烫伤','副驾座椅_缺失','副驾座椅_破损','副驾座椅_磨损','副驾座椅_老化','副驾座椅_划痕','副驾座椅_故障','副驾座椅_改装','副驾坐垫下方铁皮_正常','副驾遮阳板_脏污','副驾遮阳板_缺失','副驾遮阳板_破损','副驾阅读灯_缺失','副驾阅读灯_破损','副驾阅读灯_故障','副驾驶脚垫发霉_','副驾化妆镜灯_故障','副驾化妆镜_缺失','副驾化妆镜_破损','副驾调节方式_故障','副驾安全带卡扣_缺失','副驾安全带卡扣_破损','副驾安全带卡扣_故障','副驾安全带_脏污','副驾安全带_新更换','副驾安全带_水印','副驾安全带_破损','副驾安全带_故障','扶手箱_脏污','扶手箱_饰板丢失','扶手箱_破损','扶手箱_磨损','扶手箱_划痕','扶手箱_故障','扶手箱_改装','分动箱_异响','防冻液_液位异常','防冻液_液体变质','方向盘加热_故障','方向盘功能键_破损','方向盘功能键_磨损','方向盘功能键_故障','方向盘功能键_改装','方向盘调节开关_破损','方向盘调节开关_故障','方向盘调节方式_故障','方向盘_重新包皮','方向盘_脏污','方向盘_异响','方向盘_破损','方向盘_磨损','方向盘_改装','方向盘_沉重','发动机罩本体总成_电泳','发动机运转_不平稳','发动机油底壳_托底','发动机油底壳_破损','发动机油底壳_剐蹭','发动机油底壳_变形','发动机下护板_水箱下护板更换','发动机下护板_缺失','发动机下护板_破损','发动机上护罩_缺失','发动机上护罩_破损','发动机上护板_螺丝锈蚀','发动机漏油_异响','发动机漏油_抖动异响','发动机漏油_抖动','发动机机铰_锈蚀','发动机灯_故障','发动机舱左后_碳化','发动机舱右后_碳化','发动机舱右侧_线束破损','发动机舱盖支撑杆_缺失','发动机舱盖支撑杆_破损','发动机舱盖支撑杆_漏油','发动机舱盖支撑杆_故障','发动机舱盖支撑杆_改装','发动机舱盖锁_破损','发动机舱盖锁_故障','发动机舱盖锁_改装','发动机舱盖内衬_重做漆','发动机舱盖内衬_锈蚀','发动机舱盖内衬_胶体异常','发动机舱盖内衬_划痕','发动机舱盖内衬_封边不整','发动机舱盖内衬_变形','发动机舱盖内衬_钣金修复','发动机舱盖内边缘_重做漆','发动机舱盖内边缘_锈蚀','发动机舱盖内边缘_胶体异常','发动机舱盖内边缘_封边不整','发动机舱盖内边缘_变形','发动机舱盖内边缘_钣金修复','发动机舱盖密封条_缺失','发动机舱盖密封条_破损','发动机舱盖密封条_改装','发动机舱盖密封条_飞漆','发动机舱盖开关_缺失','发动机舱盖开关_破损','发动机舱盖开关_故障','发动机舱盖铰链_重做漆','发动机舱盖铰链_锈蚀','发动机舱盖铰链_新更换','发动机舱盖铰链_破损','发动机舱盖铰链_螺丝松动','发动机舱盖铰链_螺丝拆卸','发动机舱盖铰链_变形','发动机舱盖铰链_钣金修复','发动机舱盖隔音棉_烟熏','发动机舱盖隔音棉_缺失','发动机舱盖隔音棉_破损','发动机舱盖隔音棉_老化','发动机舱盖隔音棉_改装','发动机舱盖板_卡扣丢失','发动机舱盖_重做漆','发动机舱盖_脏污','发动机舱盖_锈蚀','发动机舱盖_新更换','发动机舱盖_色差','发动机舱盖_腻子开裂','发动机舱盖_螺丝松动','发动机舱盖_螺丝拆卸','发动机舱盖_流漆','发动机舱盖_划痕','发动机舱盖_剐蹭','发动机舱盖_改装','发动机舱盖_掉漆','发动机舱盖_变形','发动机舱盖_爆漆','发动机舱盖_雹子坑','发动机舱_脏污','发动机舱_格外清洁','发动机舱_改装','发动机_脏污','发动机_油液渗漏','发动机_油改气','发动机_异响','发动机_严重异响','发动机_严重蓝烟','发动机_新更换','发动机_维修痕迹','发动机_水温高','发动机_冒蓝烟','发动机_冒黑烟','发动机_冒白烟','发动机_螺丝松动','发动机_漏油','发动机_胶体异常','发动机_故障','发动机_改装','发动机_抖动','发电机_异响','发电机_空调压缩机更换','发电机_故障','陡坡缓降_故障','动力电池(壳体)_螺丝松动','动力电池(壳体)_划伤','动力电池(壳体)_变形','定速巡航_故障','电子稳定性控制器_提示检修','电子扇_冷凝器','电瓶故灯_故障','电动吸合门_故障','电动后视镜开关破损_故障','电动反光镜调节按钮破损_缺失','电动侧滑门_故障','电窗控制器破损_故障','电池失效_电池更换','电池底座锈迹_电池已更换','电池_新更换','点烟器底座_正常','点烟器底座_锈蚀','点烟器底座_泥沙','点烟器_缺失','点烟器_改装','点火开关_破损','点火开关_磨损','点火开关_故障','第三排座椅_脏污','第三排座椅_破损','第三排座椅_磨损','底盘护板_破损','底盘_异响','底盘_锈蚀','底盘_托底','底盘_变形','地胶（毯）_正常','地胶（毯）_脏污','地胶（毯）_水印','地胶（毯）_破损','地胶（毯）_泥沙','地胶（毯）_进水','倒车影像_故障','倒车雷达_故障','导航_故障','档把磨损_护罩破损','档把边饰板划痕_掉漆','挡把手柄_脏污','挡把手柄_破损','挡把手柄_磨损','挡把手柄_改装','挡把饰板破损_加装锁','挡把防尘套_脏污','挡把防尘套_破损','大灯清洗_故障','储物盒_缺失','储物盒_破损','储物盒_剐蹭','储物盒_改装','储物盒_变形','车厢内剐蹭_变形','车厢内_剐蹭','车外后视镜_色差','车外后视镜_缺失','车外后视镜_破损','车外后视镜_卡滞','车外后视镜_剐蹭','车外后视镜_故障','车外后视镜_改装','车身底板四边_正常','车身底板四边_碳化','车身底板四边_水印','车身底板四边_泥沙','车身底板四边_胶体异常','车身底板四边_胶体开裂','车身底板_重做漆','车身底板_正常','车身底板_脏污','车身底板_严重变形','车身底板_锈蚀','车身底板_碳化','车身底板_水印','车身底板_切割','车身底板_破损','车身底板_泥沙','车身底板_胶体开裂','车身底板_变形','车身底板_钣金修复','车内后视镜底座盖掉_无法固定','车内后视镜_缺失','车内后视镜_破损','车内后视镜_改装','车内储蓄盒_杯架故障','车门加装雨帘买方负责_破损','车辆通了电_但打不燃火','车架号划痕_磨损','车架号划痕_打刻较浅','车顶脏污_漆点','车顶油脂_油漆氧化','车顶压痕_喷漆处理','车顶行李架掉漆_风化','车顶行李架_掉漆','车顶篷_正常','车顶篷_脏污','车顶篷_烟头烫伤','车顶篷_水印','车顶篷_破损','车顶篷_划痕','车顶灯_改装','车顶_重做漆','车顶_虚漆','车顶_锈蚀','车顶_贴膜','车顶_色差','车顶_切割','车顶_腻子开裂','车顶_流漆','车顶_划痕','车顶_剐蹭','车顶_掉漆','车顶_变形','车顶_爆漆','车顶_雹子坑','车顶_钣金修复','车灯组合开关_破损','车灯组合开关_故障','侧滑门打开方式_故障','舱内线路及线束_老化','舱内线路及线束_改装','舱内管路_脏污','舱内管路_锈蚀','舱内管路_碳化','舱内管路_受热变形','舱内管路_破损','舱内管路_漏水','舱内管路_老化','舱内管路_改装','备胎支架_锈蚀','备胎支架_缺失','备胎支架_破损','备胎支架_磨损','备胎支架_划痕','备胎支架_剐蹭','备胎支架_改装','备胎支架_变形','备胎水泡_锈蚀','备胎盖饰板_脏污','备胎盖饰板_缺失','备胎盖饰板_破损','备胎盖饰板_改装','备胎舱_脏污','备胎舱_锈蚀','备胎舱_水印','备胎舱_缺失','备胎舱_破损','备胎舱_剐蹭','备胎舱_改装','备胎舱_变形','备胎_缺失','备胎_破损','备胎_磨损','杯架_缺失','杯架_破损','保养记录显示_更换钣金','保险丝盒杂乱_下沉','保险丝盒_正常','保险丝盒_水印','保险丝盒_破损','保险丝盒_泥沙','保险丝_加装','保险记录_新更换','保险记录_维修','保险杠内测_损坏','保险报案左前有碰撞_检测没问题','半轴_油底壳更换','钣金修复_不完善','钣金_新更换','钣金_喷漆','按键卡滞_不回弹','安全气囊报警_副驾驶安全气囊插头未插','Trc_off','EPC_灯故障','blis_故障','b_柱饰板已处理','AuTo_HoLd故障，按下无反应。','ABS灯_故障','ABS_故障']


In [65]:
data3m_check = data3m_check.loc[~data3m_check['检测明细描述'].isnull()]
## 检测缺失值
chek_null_count = []
# for col in fearture_columns+fearture_columns_2:
#     print(col)
#     chek_null_count.append([col,data3m_check.loc[data3m_check[col].isnull()].shape[0]/data3m_check.shape[0]])
    
for col in key_code_value:
#     print(col)
    chek_null_count.append([col,data3m_check[col].sum(),data3m_check[col].sum()/data3m_check.shape[0]])

chek_null_count_df = pd.DataFrame(chek_null_count,columns=['col','count','rate'])
chek_null_count_df.sort_values(by='count',ascending=False)
key_code_value_columns = list(chek_null_count_df.loc[chek_null_count_df['count'] >5]['col'])



In [67]:
# isCarTicket
for name in ['isLicenseTag','isStandbyKey','isSurtaxCard','isCheckVoucher']:
    data3m_check[name] = data3m_check[name].fillna("未检测")

enclosure_columns = []
for name in ['一键启动','倒车影像','倒车雷达','天窗','定速巡航','导航']:
    data3m_check.loc[data3m_check[name] == '加装',name+'_加装'] = 1
    data3m_check[name+'_加装'] = data3m_check[name+'_加装'].fillna(0)
    enclosure_columns.append(name+'_加装')

data3m_check.loc[data3m_check['主驾座椅材质'] == '改装真皮','主驾座椅材质_改装真皮'] = 1
data3m_check['主驾座椅材质_改装真皮'] = data3m_check['主驾座椅材质_改装真皮'].fillna(0) 
enclosure_columns.append('主驾座椅材质_改装真皮')           


In [68]:
for check_name in ['事故检查','其他检查','动力检查','外观检查','泡水检查','火烧检查','车内检查'  ]:
    data3m_check.loc[data3m_check[check_name].isnull(),check_name+"_缺失"] = 1 
    data3m_check[check_name+"_缺失"] = data3m_check[check_name+"_缺失"].fillna(0)
    data3m_check.loc[data3m_check[check_name]=='未见明显异常',check_name] = '0'
    data3m_check.loc[data3m_check[check_name].isnull(),check_name] = '0'
    data3m_check[check_name] = data3m_check[check_name].map(lambda x:int(x.replace('项','')))
    print(check_name,data3m_check[check_name].unique())


事故检查 [  0  10   6   2   4   5   1   3   8  11  19   7  40  24  12   9  20  16
  21  28  15  63  34  35  36  27  18  13  17  25  39  59  22  52  32  70
  14  44  30  26  91  29  57  54  49  23  42  73  31  37  53  76  33  58
  41  38  55  69  51  75  85  61  72  43  45  46  66  65 105 100 132  77
  50  71  48  56  47  90  82  67  88  81  78 159  74 153  79  99  60  87
 119 133 103  64  95  68  62  92 120  84  80 126  93]
其他检查 [ 0  3  2  1  4  7  5  6  8 10 11  9 13 19 18 50 27 35 15 12 14 23 66 24
 28 21 47 40 20 22 38 56 17 16 32 34 54 48 30 45 33 31 26 43 29 65 82 42
 36 41 25 39 71 68 52 53 44 37 51 55 60 61 57 49 62 84]
动力检查 [ 0  2  3  4  5  1  6  7 10  8  9 13 15 22 17 14 11 12 16]
外观检查 [  0  59  82  33  30  57  28  47  14  34  38  39  32  81  41  12  56  51
  43  72  60  36  55 113  37  71  73  35  54  45  25  31  17  24  42  96
  20  65  76  66  88  67 153  63  44  16  89  29  40  27  53  85  86  94
  84 128  50  26 130  62 117 115 118 106  91  70 107 104 100  80 132 102
  46  22

In [69]:
# purpose_y:'非营运', '租赁', '不详', '营转非', '营运', '教练车', '家庭用车', '预约出租客运',
#        '出租车', '其他', '预约出租客运转非', '公路货运'
# nowUsed:nan, '融资租赁公司', '个人', '汽车租赁公司', '单位使用车辆', '未检测', '汽车相关类公司', '不详',
#        '汽车销售(服务)公司', '汽车销售（服务）公司', '使用方不详', '驾校', '汽车出租公司'
# oldUsed:nan, '个人', '单位使用车辆', '汽车租赁公司', '汽车销售(服务)公司', '汽车相关类公司', '融资租赁公司',
#        '汽车销售（服务）公司', '不详', '汽车出租公司', '未检测', '驾校'

for check_name in ['nowUsed','oldUsed']:
    data3m_check[check_name] = data3m_check[check_name].fillna("未知")
    data3m_check.loc[data3m_check[check_name].isin(['汽车销售(服务)公司','汽车销售（服务）公司','汽车租赁公司','汽车相关类公司']),check_name] = '汽车公司'    
    data3m_check.loc[data3m_check[check_name].isin(['不详','使用方不详','未检测']),check_name] = '未知'
    print(check_name,data3m_check[check_name].unique())


nowUsed ['未知' '个人' '单位使用车辆' '汽车公司' '融资租赁公司' '汽车出租公司' '驾校']
oldUsed ['未知' '个人' '单位使用车辆' '汽车公司' '汽车出租公司' '驾校' '融资租赁公司']


In [70]:
check_report_org = ['isLicenseTag','isStandbyKey','isSurtaxCard','isCheckVoucher','nowUsed','oldUsed']
columns_dummies = pd.get_dummies(data3m_check[check_report_org])
check_report_columns = list(columns_dummies.columns)
data3m_check = pd.concat([data3m_check,columns_dummies],axis=1)


In [73]:
fearture_columns_1  = ['color_name_其他', 'color_name_多色', 'color_name_棕', 'color_name_灰', 'color_name_白', 'color_name_粉', 'color_name_紫', 'color_name_红', 'color_name_绿', 'color_name_蓝', 'color_name_银', 'color_name_黄', 'color_name_黑', 
                       'tradetimes_-1', 'tradetimes_0', 'tradetimes_1', 'tradetimes_10', 'tradetimes_2', 'tradetimes_3', 'tradetimes_4', 'tradetimes_5', 'tradetimes_6', 'tradetimes_7', 'tradetimes_8', 'tradetimes_9', 
                       'purpose_10', 'purpose_20', 'purpose_30', 'purpose_40', 'purpose_50']
fearture_columns_3 = ['事故检查','其他检查','动力检查','外观检查','泡水检查','火烧检查','车内检查'  ] + [ col+"_缺失" for  col in ['事故检查','其他检查','动力检查','外观检查','泡水检查','火烧检查','车内检查'  ] ]


In [75]:
"""
    增加交强险和保险相关信息
"""

'\n    增加交强险和保险相关信息\n'

In [76]:
def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        return 0 
    
data3m_check['insure_days'] = list(map(lambda startTime,endTime:date_time_sub(startTime,endTime,"%Y-%m-%d"),data3m_check['pay_time'],data3m_check['insureDate']))
data3m_check.loc[data3m_check['insure_days'] <=0 ,'insure_days'] = 0
data3m_check['insure_days'] = data3m_check['insure_days'].fillna(0)
data3m_check.loc[data3m_check['insureDate'].isnull(),'insure_days_q'] = 1
data3m_check['insure_days_q'] = data3m_check['insure_days_q'].fillna(0)

data3m_check['inspection_days'] = list(map(lambda startTime,endTime:date_time_sub(startTime,endTime,"%Y-%m-%d"),data3m_check['pay_time'],data3m_check['inspectionDate']))
data3m_check.loc[data3m_check['inspection_days'] <=0 ,'inspection_days'] = 0
data3m_check['inspection_days'] = data3m_check['inspection_days'].fillna(0)
data3m_check.loc[data3m_check['inspectionDate'].isnull(),'inspection_days_q'] = 1
data3m_check['inspection_days_q'] = data3m_check['inspection_days_q'].fillna(0)
data3m_check['insure_days'] = data3m_check['insure_days'].map(lambda x:round(math.log(x+1,100),2))
data3m_check['insure_days'] = data3m_check['insure_days'].map(lambda x:round(math.log(x+1,100),2))


In [78]:
"""
    增加报废相关信息
"""

'\n    增加报废相关信息\n'

In [79]:
data3m_check['车况提示_买方负责'] = data3m_check['车况提示'] + data3m_check['买方负责']


In [80]:
import math
def get_excep(x,str_name):
    if str_name in str(x) :
        return str_name

def get_strtime(text):
    import re
    text = text.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").replace(".", "-").strip()
    text = re.sub("\s+", " ", text)
    t = ""
    regex_list = [
    # 2013年8月15日 22:46:21
       "(\d{4}-\d{1,2}-\d{1,2} \d{1,2}:\d{1,2}:\d{1,2})",
       # "2013年8月15日 22:46"
       "(\d{4}-\d{1,2}-\d{1,2} \d{1,2}:\d{1,2})",
       # "2014年5月11日"
       "(\d{4}-\d{1,2}-\d{1,2})",
       # "2014年5月"
       "(\d{4}-\d{1,2})",
    ]
    for regex in regex_list:
        t = re.search(regex, text)
        if t:
            t = t.group(1)
            return t
        else:
            pass
#             print("没有获取到有效日期")
         
    return t

data3m_check['强制报废'] = data3m_check['车况提示_买方负责'].map(lambda x:get_excep(x,'强制报废'))
data3m_check['报废时间'] = data3m_check['车况提示_买方负责'].map(lambda x:get_strtime(str(x)))

data3m_check['baofei_days'] = list(map(lambda startTime,endTime:date_time_sub(startTime,endTime,"%Y-%m-%d"),data3m_check['pay_time'],data3m_check['报废时间']))
data3m_check.loc[data3m_check['baofei_days'] <=0 ,'baofei_days'] = 0 
data3m_check.loc[data3m_check['强制报废'] == '强制报废','baofei_if'] = 1 
data3m_check['baofei_if'] = data3m_check['baofei_if'].fillna(0)
data3m_check['baofei_days'] = data3m_check['baofei_days'].fillna(0)
data3m_check['baofei_days'] = data3m_check['baofei_days'].map(lambda x:round(math.log(x+1,100),2))


In [81]:
"""
    增加其他信息
"""

'\n    增加其他信息\n'

In [82]:
other_columns = []
for col  in ['轮胎规格与登记证不符','实车车架号未见','过户票缺失','铭牌生产日期与登记证不一致','同轴轮胎花纹不一致',
            '补领登记证一次','保险人与被保险人不一致','车船税未见（存在欠税可能）','车牌所在地与交强险所在地不一致',
            '购车发票缺失','实车公里数不符','变更登记','交强险到期时间未见','年检过期或即将过',
            '交强险已过期或即将过期','公里数与实车不符','安全带缺失','加装遮雨板','车身抖动','加装雨挡','年检本月到期','轮毂改装','车辆改装触摸导航大屏，加装雨挡',
            '60万公里强制报废','使用性质预约出租客运','维保公里数异常','需要面签','年检再过户',
            '加装雨眉，加装尾翼','有改装','加装踏板','此车已参加BJEV主机厂二手车置换政策，不再享受免费的技术升级、保修保养、动力电池总成质保']:
    data3m_check[col] = data3m_check['车况提示_买方负责'].map(lambda x:get_excep(x,col))
    print(col,data3m_check.loc[data3m_check[col] == col].shape[0])
    data3m_check.loc[data3m_check[col] == col,col] = 1 
    data3m_check[col] = data3m_check[col].fillna(0)
    other_columns.append(col)

fearture_columns_4 = ['baofei_days','baofei_if','insure_days','insure_days_q','inspection_days','inspection_days_q']
data3m_check['公里数与实车不符'] = data3m_check['公里数与实车不符']  + data3m_check['实车公里数不符']
data3m_check.loc[data3m_check['公里数与实车不符'] == 2,'公里数与实车不符'] = 1
other_columns = ['轮胎规格与登记证不符','过户票缺失','同轴轮胎花纹不一致','保险人与被保险人不一致','车牌所在地与交强险所在地不一致',
                '购车发票缺失','交强险到期时间未见','年检过期或即将过','有改装','交强险已过期或即将过期','车身抖动',
                '车船税未见（存在欠税可能）','实车车架号未见','公里数与实车不符']


轮胎规格与登记证不符 19525
实车车架号未见 1279
过户票缺失 12748
铭牌生产日期与登记证不一致 2710
同轴轮胎花纹不一致 12227
补领登记证一次 1219
保险人与被保险人不一致 32337
车船税未见（存在欠税可能） 31929
车牌所在地与交强险所在地不一致 2679
购车发票缺失 19755
实车公里数不符 610
变更登记 6319
交强险到期时间未见 5425
年检过期或即将过 17736
交强险已过期或即将过期 13741
公里数与实车不符 7760
安全带缺失 143
加装遮雨板 13
车身抖动 731
加装雨挡 3361
年检本月到期 23
轮毂改装 1532
车辆改装触摸导航大屏，加装雨挡 1
60万公里强制报废 10
使用性质预约出租客运 2
维保公里数异常 53
需要面签 11841
年检再过户 762
加装雨眉，加装尾翼 2
有改装 24100
加装踏板 1458
此车已参加BJEV主机厂二手车置换政策，不再享受免费的技术升级、保修保养、动力电池总成质保 168


In [ ]:
data3m_check['days'] = data3m_check['days'].map(lambda x:round(np.log(x),2))


In [85]:
data3m_check = data3m_check.loc[~data3m_check[label].isnull()]
data3m_check = data3m_check.loc[data3m_check['order_status'] == 80]
data3m_check = data3m_check.loc[data3m_check[label]<0.99]


In [88]:
data3m_check = data3m_check.loc[(~data3m_check['series_name_'].isnull()) & 
                (data3m_check['series_name'] == data3m_check['series_name_'])]


In [89]:
## 剔除弹个车店铺的车辆
data3m_check = data3m_check.loc[~data3m_check[label].isnull()]
test_data = data3m_check.loc[(data3m_check['pay_time']>='2020-08-01')& 
                        (data3m_check['pay_time']<'2020-10-01')&
                        (data3m_check['tenant_name'] != '弹个车拍卖店铺') & 
                        (data3m_check['seller_name'] != '弹个车线上测试店铺') & 
                        (data3m['auction_mode'].isin(['2','4'])) ]

train_data = data3m_check.loc[(data3m_check['pay_time']<'2020-08-01') & 
                        (data3m_check['pay_time']>'2016-01-01')&
                        (data3m_check['tenant_name'] != '弹个车拍卖店铺') & 
                        (data3m_check['seller_name'] != '弹个车线上测试店铺')]


In [90]:
print('训练集数据量：',train_data.shape,'测试集数据量：',test_data.shape)


训练集数据量： (154104, 2999) 测试集数据量： (5921, 2999)


In [91]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 5000,
                        max_depth = 10,
                        num_leaves = 200,
                        colsample_bytree = 0.22, 
                        min_child_samples = 110, 
                        n_jobs = 8,
                        verbose = 1)


In [121]:
## 定义特征
key_code_value_columns_new = []
for col in key_code_value_columns:
    if '\n' in col and len(col) > 5:
        print('带\n:',col)
    else:
        key_code_value_columns_new.append(col)
    if ' ' in col :
        print("带空格：",col)
fearture_columns = ['month', 'model_year', 'license_month_log', 'rate', 'rate_count', 'per_mile', 'year_err', 
                    'province_name0', 'province_name1', 'province_name2', 'province_name3', 'province_name4', 'province_name5', 'province_name6', 'province_name7', 'province_name8', 'province_name9', 'province_name10', 'province_name11', 'province_name12', 'province_name13', 'province_name14', 'province_name15', 'province_name16', 'province_name17', 'province_name18', 'province_name19', 'province_name20', 'province_name21', 'province_name22', 'province_name23', 'province_name24', 'province_name25', 'province_name26', 'province_name27', 'province_name28', 'province_name29', 'province_name30', 
                    'keep_value)_log', 'quality', 'new_car_price_log', 'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'cylinder_number', 'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 
                    'country_id0', 'country_id1', 'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3', 'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 
                    'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29', 'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35', 'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 'series_level60', 'series_level61', 
                    'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14', 'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23', 'wordvec24', 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41', 'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50', 'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 'wordvec60', 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77', 'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86', 'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 'wordvec96', 'wordvec97', 'wordvec98', 'wordvec99']

fearture_columns_model = fearture_columns+fearture_columns_1+check_report_columns+\
                            enclosure_columns+key_code_value_columns_new+fearture_columns_4 + other_columns
print('特征总数量：',len(fearture_columns_model))


特征总数量： 2218


In [116]:
gbm.fit(train_data[fearture_columns_model], train_data[label],
        eval_set = (test_data[fearture_columns_model], test_data[label]),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)


[1]	valid_0's l1: 0.118909	valid_0's l2: 0.0232672
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l1: 0.115746	valid_0's l2: 0.0221109
[3]	valid_0's l1: 0.112624	valid_0's l2: 0.0209701
[4]	valid_0's l1: 0.109705	valid_0's l2: 0.0199476
[5]	valid_0's l1: 0.106807	valid_0's l2: 0.0189546
[6]	valid_0's l1: 0.105003	valid_0's l2: 0.0183518
[7]	valid_0's l1: 0.102171	valid_0's l2: 0.0174173
[8]	valid_0's l1: 0.100542	valid_0's l2: 0.0168987
[9]	valid_0's l1: 0.0990242	valid_0's l2: 0.0164499
[10]	valid_0's l1: 0.0977549	valid_0's l2: 0.0160408
[11]	valid_0's l1: 0.096309	valid_0's l2: 0.0156036
[12]	valid_0's l1: 0.0938317	valid_0's l2: 0.0148453
[13]	valid_0's l1: 0.0913993	valid_0's l2: 0.0141191
[14]	valid_0's l1: 0.0901882	valid_0's l2: 0.0137687
[15]	valid_0's l1: 0.0881094	valid_0's l2: 0.0131336
[16]	valid_0's l1: 0.0859417	valid_0's l2: 0.0125262
[17]	valid_0's l1: 0.0848876	valid_0's l2: 0.0122472
[18]	valid_0's l1: 0.0829352	valid_0's l2: 0.0116794
[1

[153]	valid_0's l1: 0.0285636	valid_0's l2: 0.00162545
[154]	valid_0's l1: 0.0285052	valid_0's l2: 0.00161941
[155]	valid_0's l1: 0.0284875	valid_0's l2: 0.00161773
[156]	valid_0's l1: 0.0284176	valid_0's l2: 0.00161055
[157]	valid_0's l1: 0.0283494	valid_0's l2: 0.00160354
[158]	valid_0's l1: 0.0283332	valid_0's l2: 0.0016019
[159]	valid_0's l1: 0.0282595	valid_0's l2: 0.00159395
[160]	valid_0's l1: 0.0281973	valid_0's l2: 0.00158705
[161]	valid_0's l1: 0.0281438	valid_0's l2: 0.00158095
[162]	valid_0's l1: 0.0281196	valid_0's l2: 0.00157817
[163]	valid_0's l1: 0.0280571	valid_0's l2: 0.00157167
[164]	valid_0's l1: 0.0280003	valid_0's l2: 0.00156504
[165]	valid_0's l1: 0.0279473	valid_0's l2: 0.00155825
[166]	valid_0's l1: 0.0278939	valid_0's l2: 0.00155245
[167]	valid_0's l1: 0.0278696	valid_0's l2: 0.0015502
[168]	valid_0's l1: 0.0278503	valid_0's l2: 0.00154797
[169]	valid_0's l1: 0.0278376	valid_0's l2: 0.00154668
[170]	valid_0's l1: 0.0278071	valid_0's l2: 0.00154296
[171]	valid_

[304]	valid_0's l1: 0.0246348	valid_0's l2: 0.00122155
[305]	valid_0's l1: 0.0246237	valid_0's l2: 0.00122038
[306]	valid_0's l1: 0.0246132	valid_0's l2: 0.00121941
[307]	valid_0's l1: 0.0245902	valid_0's l2: 0.00121707
[308]	valid_0's l1: 0.0245753	valid_0's l2: 0.00121596
[309]	valid_0's l1: 0.0245499	valid_0's l2: 0.0012138
[310]	valid_0's l1: 0.0245299	valid_0's l2: 0.00121212
[311]	valid_0's l1: 0.0245106	valid_0's l2: 0.00121032
[312]	valid_0's l1: 0.0244939	valid_0's l2: 0.00120868
[313]	valid_0's l1: 0.0244803	valid_0's l2: 0.00120717
[314]	valid_0's l1: 0.0244609	valid_0's l2: 0.00120548
[315]	valid_0's l1: 0.0244445	valid_0's l2: 0.00120394
[316]	valid_0's l1: 0.0244248	valid_0's l2: 0.00120201
[317]	valid_0's l1: 0.0244125	valid_0's l2: 0.001201
[318]	valid_0's l1: 0.0243934	valid_0's l2: 0.00119936
[319]	valid_0's l1: 0.0243756	valid_0's l2: 0.00119777
[320]	valid_0's l1: 0.0243618	valid_0's l2: 0.00119688
[321]	valid_0's l1: 0.0243579	valid_0's l2: 0.00119636
[322]	valid_0

[454]	valid_0's l1: 0.0231357	valid_0's l2: 0.00108726
[455]	valid_0's l1: 0.023132	valid_0's l2: 0.00108701
[456]	valid_0's l1: 0.0231278	valid_0's l2: 0.00108671
[457]	valid_0's l1: 0.0231247	valid_0's l2: 0.00108652
[458]	valid_0's l1: 0.023115	valid_0's l2: 0.00108574
[459]	valid_0's l1: 0.0231116	valid_0's l2: 0.00108541
[460]	valid_0's l1: 0.0231069	valid_0's l2: 0.00108502
[461]	valid_0's l1: 0.0231018	valid_0's l2: 0.00108441
[462]	valid_0's l1: 0.0230889	valid_0's l2: 0.00108327
[463]	valid_0's l1: 0.0230894	valid_0's l2: 0.00108321
[464]	valid_0's l1: 0.0230877	valid_0's l2: 0.00108319
[465]	valid_0's l1: 0.0230746	valid_0's l2: 0.00108243
[466]	valid_0's l1: 0.0230658	valid_0's l2: 0.0010815
[467]	valid_0's l1: 0.0230484	valid_0's l2: 0.00108015
[468]	valid_0's l1: 0.0230392	valid_0's l2: 0.00107918
[469]	valid_0's l1: 0.0230277	valid_0's l2: 0.00107821
[470]	valid_0's l1: 0.0230217	valid_0's l2: 0.00107762
[471]	valid_0's l1: 0.0230204	valid_0's l2: 0.00107746
[472]	valid_0

[606]	valid_0's l1: 0.0223942	valid_0's l2: 0.00102218
[607]	valid_0's l1: 0.0223915	valid_0's l2: 0.00102205
[608]	valid_0's l1: 0.0223865	valid_0's l2: 0.00102167
[609]	valid_0's l1: 0.0223808	valid_0's l2: 0.00102087
[610]	valid_0's l1: 0.0223689	valid_0's l2: 0.00101967
[611]	valid_0's l1: 0.0223649	valid_0's l2: 0.00101927
[612]	valid_0's l1: 0.022363	valid_0's l2: 0.00101914
[613]	valid_0's l1: 0.0223568	valid_0's l2: 0.00101839
[614]	valid_0's l1: 0.0223542	valid_0's l2: 0.00101822
[615]	valid_0's l1: 0.0223502	valid_0's l2: 0.00101758
[616]	valid_0's l1: 0.0223451	valid_0's l2: 0.00101719
[617]	valid_0's l1: 0.022342	valid_0's l2: 0.00101682
[618]	valid_0's l1: 0.0223392	valid_0's l2: 0.00101649
[619]	valid_0's l1: 0.0223293	valid_0's l2: 0.00101539
[620]	valid_0's l1: 0.0223234	valid_0's l2: 0.00101468
[621]	valid_0's l1: 0.0223219	valid_0's l2: 0.00101461
[622]	valid_0's l1: 0.0223196	valid_0's l2: 0.00101447
[623]	valid_0's l1: 0.0223134	valid_0's l2: 0.00101396
[624]	valid_

...

[2662]	valid_0's l1: 0.0200226	valid_0's l2: 0.000816502
[2663]	valid_0's l1: 0.020023	valid_0's l2: 0.000816522
[2664]	valid_0's l1: 0.0200228	valid_0's l2: 0.00081647
[2665]	valid_0's l1: 0.0200222	valid_0's l2: 0.000816394
[2666]	valid_0's l1: 0.0200221	valid_0's l2: 0.000816376
[2667]	valid_0's l1: 0.0200215	valid_0's l2: 0.000816335
[2668]	valid_0's l1: 0.0200216	valid_0's l2: 0.000816333
[2669]	valid_0's l1: 0.0200215	valid_0's l2: 0.000816322
[2670]	valid_0's l1: 0.0200205	valid_0's l2: 0.000816246
[2671]	valid_0's l1: 0.0200195	valid_0's l2: 0.000816144
[2672]	valid_0's l1: 0.0200191	valid_0's l2: 0.000816126
[2673]	valid_0's l1: 0.0200189	valid_0's l2: 0.000816151
[2674]	valid_0's l1: 0.0200199	valid_0's l2: 0.000816126
[2675]	valid_0's l1: 0.0200197	valid_0's l2: 0.00081612
[2676]	valid_0's l1: 0.0200183	valid_0's l2: 0.000816057
[2677]	valid_0's l1: 0.0200184	valid_0's l2: 0.000816053
[2678]	valid_0's l1: 0.0200185	valid_0's l2: 0.00081604
[2679]	valid_0's l1: 0.0200172	vali

[2809]	valid_0's l1: 0.0199813	valid_0's l2: 0.000813186
[2810]	valid_0's l1: 0.0199812	valid_0's l2: 0.000813248
[2811]	valid_0's l1: 0.0199812	valid_0's l2: 0.00081325
[2812]	valid_0's l1: 0.0199792	valid_0's l2: 0.000813085
[2813]	valid_0's l1: 0.0199794	valid_0's l2: 0.000813097
[2814]	valid_0's l1: 0.0199785	valid_0's l2: 0.000813053
[2815]	valid_0's l1: 0.0199781	valid_0's l2: 0.000813032
[2816]	valid_0's l1: 0.0199775	valid_0's l2: 0.000812965
[2817]	valid_0's l1: 0.0199759	valid_0's l2: 0.000812864
[2818]	valid_0's l1: 0.0199761	valid_0's l2: 0.00081287
[2819]	valid_0's l1: 0.0199755	valid_0's l2: 0.000812819
[2820]	valid_0's l1: 0.0199758	valid_0's l2: 0.000812828
[2821]	valid_0's l1: 0.0199756	valid_0's l2: 0.000812815
[2822]	valid_0's l1: 0.019976	valid_0's l2: 0.000812828
[2823]	valid_0's l1: 0.0199755	valid_0's l2: 0.000812763
[2824]	valid_0's l1: 0.0199733	valid_0's l2: 0.000812587
[2825]	valid_0's l1: 0.0199733	valid_0's l2: 0.000812593
[2826]	valid_0's l1: 0.0199731	val

[2956]	valid_0's l1: 0.0199481	valid_0's l2: 0.000810376
[2957]	valid_0's l1: 0.0199479	valid_0's l2: 0.000810363
[2958]	valid_0's l1: 0.0199472	valid_0's l2: 0.000810337
[2959]	valid_0's l1: 0.0199472	valid_0's l2: 0.000810314
[2960]	valid_0's l1: 0.0199468	valid_0's l2: 0.000810313
[2961]	valid_0's l1: 0.0199469	valid_0's l2: 0.000810316
[2962]	valid_0's l1: 0.019947	valid_0's l2: 0.000810317
[2963]	valid_0's l1: 0.019947	valid_0's l2: 0.000810303
[2964]	valid_0's l1: 0.0199474	valid_0's l2: 0.000810335
[2965]	valid_0's l1: 0.0199473	valid_0's l2: 0.000810338
[2966]	valid_0's l1: 0.0199479	valid_0's l2: 0.000810341
[2967]	valid_0's l1: 0.0199483	valid_0's l2: 0.000810353
[2968]	valid_0's l1: 0.0199474	valid_0's l2: 0.00081033
[2969]	valid_0's l1: 0.0199472	valid_0's l2: 0.000810316
[2970]	valid_0's l1: 0.0199465	valid_0's l2: 0.00081031
[2971]	valid_0's l1: 0.0199454	valid_0's l2: 0.000810245
[2972]	valid_0's l1: 0.0199452	valid_0's l2: 0.000810196
[2973]	valid_0's l1: 0.019945	valid

[3103]	valid_0's l1: 0.019904	valid_0's l2: 0.000806936
[3104]	valid_0's l1: 0.0199036	valid_0's l2: 0.000806891
[3105]	valid_0's l1: 0.019903	valid_0's l2: 0.00080684
[3106]	valid_0's l1: 0.0199042	valid_0's l2: 0.000806887
[3107]	valid_0's l1: 0.0199048	valid_0's l2: 0.000806925
[3108]	valid_0's l1: 0.0199047	valid_0's l2: 0.000806936
[3109]	valid_0's l1: 0.0199048	valid_0's l2: 0.000806949
[3110]	valid_0's l1: 0.0199047	valid_0's l2: 0.000806889
[3111]	valid_0's l1: 0.0199047	valid_0's l2: 0.000806903
[3112]	valid_0's l1: 0.0199041	valid_0's l2: 0.00080687
[3113]	valid_0's l1: 0.019904	valid_0's l2: 0.000806864
[3114]	valid_0's l1: 0.0199041	valid_0's l2: 0.000806878
[3115]	valid_0's l1: 0.0199042	valid_0's l2: 0.000806893
[3116]	valid_0's l1: 0.0199048	valid_0's l2: 0.000806919
[3117]	valid_0's l1: 0.0199049	valid_0's l2: 0.000806945
[3118]	valid_0's l1: 0.0199051	valid_0's l2: 0.000806965
[3119]	valid_0's l1: 0.0199053	valid_0's l2: 0.000806959
[3120]	valid_0's l1: 0.0199052	valid

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.22,
       learning_rate=0.03, max_depth=10, min_child_samples=110,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=5000,
       n_jobs=8, num_leaves=200, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [118]:
feature_importances_df = pd.DataFrame(fearture_columns_model,columns=['fearture'])
feature_importances_df['feature_importances'] = gbm.feature_importances_

feature_importances_df.sort_values(by='feature_importances',ascending=False)
delete_columns = list(feature_importances_df.loc[feature_importances_df['feature_importances'] == 0]['fearture'])

feature_importances_df.sort_values(by='feature_importances',ascending=False)

,fearture,feature_importances
38,keep_value,7090
2,license_month,7038
40,new_car_price_log,4693
5,per_mile,4597
4,rate_count,3043
1,model_year,3042
43,height,2845
42,length,2645
0,days_log,2561
3,rate,2545


In [119]:
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))

from sklearn.metrics import r2_score
test_data['predict_0'] = gbm.predict(test_data[fearture_columns_model].fillna(0))
test_t2 = r2_score(test_data['residual'],test_data['predict_0'])
train_data['predict_0'] = gbm.predict(train_data[fearture_columns_model].fillna(0))
train_t2 = r2_score(train_data['residual'],train_data['predict_0'])

test_data['差异'] = round((test_data['predict_0']-test_data['residual'])/test_data['residual'],2)
train_data['差异'] = round((train_data['predict_0']-train_data['residual'])/train_data['residual'],2)

# print("   筛选后的数据量：",data3m_series_sales.shape)
print('训练集和测试集决定系数：',train_t2,test_t2)
logger.log('车况明细模型训练集和测试集决定系数：%s %s' % (train_t2,test_t2))
print("测试集评估。。。")
logger.log('车况明细模型 测试数据评估。。。。。')
data3m_pinggu(test_data,'差异')
print("训练集评估。。。")
logger.log('车况明细模型 训练数据评估。。。。。')
data3m_pinggu(train_data,'差异')

# P<3%:  0.3254
# P<5%:  0.4804
# P<8%:  0.6668
# P<10%:  0.7489
# P<20%:  0.9293
# 训练集评估。。。
# [2020-10-22 15:53:01,454] {<ipython-input-315-18214e15c7d3>:85} INFO - 车况明细模型 训练数据评估。。。。。
# P<3%:  0.4646
# P<5%:  0.6418
# P<8%:  0.8023
# P<10%:  0.8629
# P<20%:  0.9685

# P<3%:  0.3298
# P<5%:  0.4844
# P<8%:  0.6649
# P<10%:  0.7521
# P<20%:  0.9303
# 训练集评估。。。
# [2020-10-22 16:04:15,817] {<ipython-input-315-18214e15c7d3>:85} INFO - 车况明细模型 训练数据评估。。。。。
# P<3%:  0.4829
# P<5%:  0.6611
# P<8%:  0.8163
# P<10%:  0.8736
# P<20%:  0.9712

# P<3%:  0.3215
# P<5%:  0.4828
# P<8%:  0.6719
# P<10%:  0.7554
# P<20%:  0.93
# 训练集评估。。。
# [2020-10-22 16:26:48,275] {<ipython-input-315-18214e15c7d3>:85} INFO - 车况明细模型 训练数据评估。。。。。
# P<3%:  0.5139
# P<5%:  0.6909
# P<8%:  0.8372
# P<10%:  0.8902
# P<20%:  0.9764



/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

训练集和测试集决定系数： 0.9861654859831612 0.9664327070398983
[2020-11-30 16:36:44,042] {<ipython-input-1-18214e15c7d3>:85} INFO - 车况明细模型训练集和测试集决定系数：0.9861654859831612 0.9664327070398983
测试集评估。。。
[2020-11-30 16:36:44,044] {<ipython-input-1-18214e15c7d3>:85} INFO - 车况明细模型 测试数据评估。。。。。


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


P<3%:  0.3153
P<5%:  0.4712
P<8%:  0.6496
P<10%:  0.7387
P<20%:  0.9281
训练集评估。。。
[2020-11-30 16:36:44,438] {<ipython-input-1-18214e15c7d3>:85} INFO - 车况明细模型 训练数据评估。。。。。
P<3%:  0.4418
P<5%:  0.621
P<8%:  0.7888
P<10%:  0.8535
P<20%:  0.9662


In [113]:
data3m_pinggu(test_data.loc[test_data['environmental'] != '国III'],'差异')


P<3%:  0.3346
P<5%:  0.4997
P<8%:  0.6805
P<10%:  0.7671
P<20%:  0.9389


In [ ]:
"""
    销量取2020年的销量百分比的前30%、40%、50%、60%、70%、80%
"""

In [1216]:
data3m_2020 = data3m.loc[data3m['pay_time'] >='2020-01-01']
series_name_count = data3m_2020[['model_code','brand_name']].groupby('model_code').count().\
                        reset_index().rename(columns={'brand_name':"count"})
series_name_count = series_name_count.sort_values(by='count',ascending=False)
series_name_count['累计'] = series_name_count['count'].cumsum()
series_name_count['总数'] = series_name_count['count'].sum()
series_name_count['销量百分比'] = series_name_count['累计'] /series_name_count['总数']

for num in range(3,9):
    print('销量前：  '+str(num*10)+"%")
    series_name_list = list(series_name_count.loc[series_name_count['销量百分比'] <=num/10]['model_code'].unique())
    data3m_series_sales = data3m.loc[data3m['model_code'].isin(series_name_list)]
    
    data3m_pinggu(test_data.loc[test_data['model_code'].isin(series_name_list)],'差异')
    print("训练集评估。。。")
    data3m_pinggu(train_data.loc[train_data['model_code'].isin(series_name_list)],'差异')


销量前：  30%
P<3%:  0.3594
P<5%:  0.5383
P<8%:  0.7332
P<10%:  0.8163
P<20%:  0.9633
训练集评估。。。
P<3%:  0.5084
P<5%:  0.6957
P<8%:  0.8558
P<10%:  0.9104
P<20%:  0.9854
销量前：  40%
P<3%:  0.3684
P<5%:  0.5506
P<8%:  0.7397
P<10%:  0.8218
P<20%:  0.959
训练集评估。。。
P<3%:  0.513
P<5%:  0.6991
P<8%:  0.8568
P<10%:  0.9108
P<20%:  0.9848
销量前：  50%
P<3%:  0.3715
P<5%:  0.5489
P<8%:  0.7285
P<10%:  0.811
P<20%:  0.9609
训练集评估。。。
P<3%:  0.5146
P<5%:  0.7007
P<8%:  0.8579
P<10%:  0.9109
P<20%:  0.9847
销量前：  60%
P<3%:  0.3665
P<5%:  0.5462
P<8%:  0.7264
P<10%:  0.8092
P<20%:  0.9586
训练集评估。。。
P<3%:  0.5154
P<5%:  0.7013
P<8%:  0.8564
P<10%:  0.9097
P<20%:  0.9836
销量前：  70%
P<3%:  0.3527
P<5%:  0.5306
P<8%:  0.7145
P<10%:  0.7998
P<20%:  0.9551
训练集评估。。。
P<3%:  0.5126
P<5%:  0.6975
P<8%:  0.852
P<10%:  0.9049
P<20%:  0.9822
销量前：  80%
P<3%:  0.3443
P<5%:  0.5216
P<8%:  0.7035
P<10%:  0.7908
P<20%:  0.9506
训练集评估。。。
P<3%:  0.5101
P<5%:  0.6942
P<8%:  0.8482
P<10%:  0.9018
P<20%:  0.9813


In [164]:
train_data['model_price'] = train_data['guide_price'] * train_data['predict_0']
test_data['model_price'] = test_data['guide_price'] * test_data['predict_0']


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [161]:
test_data_err_count = test_data.loc[(test_data['差异'] >=0.1) | (test_data['差异'] <=-0.1)][['model_code','order_id']].\
groupby(['model_code']).count().reset_index().rename(columns={'order_id':"test_err_count"})

test_data_count = test_data[['model_code','model_name','brand_name','order_id']].\
groupby(['model_code','model_name','brand_name']).count().reset_index().rename(columns={'order_id':"test_count"})
                                        
train_data_err_count = train_data.loc[(train_data['差异'] >=0.1) | (train_data['差异'] <=-0.1)][['model_code','order_id']].\
groupby('model_code').count().reset_index().rename(columns={'order_id':"train_err_count"})

train_data_count = train_data[['model_code','order_id']].\
groupby('model_code').count().reset_index().rename(columns={'order_id':"train_count"})


test_data_count = pd.merge(test_data_count,test_data_err_count,on='model_code',how='left')
test_data_count = pd.merge(test_data_count,train_data_err_count,on='model_code',how='left')
test_data_count = pd.merge(test_data_count,train_data_count,on='model_code',how='left')
    
test_data_count['train_rate'] = test_data_count['train_err_count'] / test_data_count['train_count']
test_data_count['test_rate'] = test_data_count['test_err_count'] / test_data_count['test_count']


In [162]:
model_code_list = list(test_data_count.loc[~((test_data_count['test_rate'] > 0.1) & (test_data_count['train_rate'] > 0.1))]['model_code'])
data3m_pinggu(test_data.loc[test_data['model_code'].isin(model_code_list)],'差异')


P<3%:  0.36
P<5%:  0.5473
P<8%:  0.7611
P<10%:  0.8471
P<20%:  0.9685
